In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import plotly.express as px
import plotly.graph_objs as go
from statsmodels.tsa.seasonal import DecomposeResult, seasonal_decompose
import statsmodels.api as sm
from scipy.stats import boxcox
from prophet import Prophet
from TSUtilities.TSTrend.trend_dampen import TrendDampen
import itertools
import warnings
warnings.filterwarnings("ignore")

/Users/adulla/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected = True)

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
traffic_orders_data = pd.read_csv('/Users/adulla/Desktop/traffic_orders_V3.csv')

In [5]:
checkout_traffic_data = pd.read_excel('/Users/adulla/Desktop/checkout_traffic.xlsx')

In [6]:
subscribers_data = pd.read_csv('/Users/adulla/Desktop/gross_new_subs.csv')

In [7]:
# specify which Geo you want to produce forecasts. 
# available options: AMER (G), ASIA (G), EMEA (G), JPN (G), ALL (G)
# ALL (G) refers to total aggregate across all geo regions 

geo = 'ALL (G)'

### Function Storing

In [8]:
def data_cleaning_traffic_orders_SARIMAX(df):
    # only Web is represented
    df_copy = df.copy()
    del df_copy['Route to Market - Subscriptions']
    # filter for Geo
    if geo == 'ALL (G)':
        cleaned_df = df_copy.groupby(['Planning DDOM-Traffic_m', 'Fiscal Week'])['Value'].sum().reset_index()
    else:
        cleaned_df = df_copy.loc[df_copy['Market Area'] == geo]
        del cleaned_df['Market Area']
        cleaned_df = cleaned_df.groupby(['Planning DDOM-Traffic_m', 'Fiscal Week'])['Value'].sum().reset_index()
    cleaned_df = cleaned_df.set_index(['Planning DDOM-Traffic_m', 'Fiscal Week']).unstack(['Planning DDOM-Traffic_m'])
    cleaned_df.columns = cleaned_df.columns.get_level_values(0)
    cleaned_df = cleaned_df.reset_index()
    cleaned_df.columns = ['Fiscal Week', 'Traffic', 'Orders']
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].astype(str)
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].str[:4] + '-' + cleaned_df['Fiscal Week'].str[4:]
    # remove 53rd week from all years (only 1 occurence)
    cleaned_df[['year', 'week']] = cleaned_df['Fiscal Week'].str.split("-", 1, expand = True)
    cleaned_df = cleaned_df[cleaned_df['week'] != '53']
    del cleaned_df['week']
    del cleaned_df['year']
    cleaned_df['Fiscal Date'] = pd.to_datetime(cleaned_df['Fiscal Week'] + '-1', format = '%Y-%W-%w')
    del cleaned_df['Fiscal Week']
    cleaned_df.index = cleaned_df['Fiscal Date']
    del cleaned_df['Fiscal Date']
    cleaned_df = cleaned_df['2019-01-07':]
    return cleaned_df

def data_cleaning_checkout_SARIMAX(df):
    df_copy = df.copy()
    del df_copy['DDOM Web Funnel Step']
    df_copy = df_copy[df_copy['Unit'] != 0].dropna()
    # filter for Geo
    if geo == 'ALL (G)':
        cleaned_df = df_copy.groupby(['Fiscal Week'])['Unit'].sum().reset_index()
    else:
        cleaned_df = df_copy.loc[df_copy['Market Area'] == geo]
        del cleaned_df['Market Area']
        cleaned_df = cleaned_df.groupby(['Fiscal Week'])['Unit'].sum().reset_index()
    cleaned_df = cleaned_df.set_index(['Fiscal Week'])
    cleaned_df.columns = cleaned_df.columns.get_level_values(0)
    cleaned_df = cleaned_df.reset_index()
    cleaned_df.columns = ['Fiscal Week', 'Checkout']
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].astype(str)
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].str[:4] + '-' + cleaned_df['Fiscal Week'].str[4:]
    cleaned_df[['year', 'week']] = cleaned_df['Fiscal Week'].str.split("-", 1, expand = True)
    cleaned_df = cleaned_df[cleaned_df['week'] != '53']
    del cleaned_df['week']
    del cleaned_df['year']
    cleaned_df['Fiscal Date'] = pd.to_datetime(cleaned_df['Fiscal Week'] + '-1', format = '%Y-%W-%w')
    del cleaned_df['Fiscal Week']
    cleaned_df.index = cleaned_df['Fiscal Date']
    del cleaned_df['Fiscal Date']
    cleaned_df = cleaned_df['2021-12-06':'2023-08-07']
    return cleaned_df

def data_cleaning_subscribers_SARIMAX(df):
    df_copy = df.copy()
    del df_copy['Planning Subscriptions_m']
    # filter for Geo
    if geo == 'ALL (G)':
        cleaned_df = df_copy.groupby(['Type', 'Fiscal Week'])['Value'].sum().reset_index()
    else:
        cleaned_df = df_copy.loc[df_copy['Market Area'] == geo]
        del cleaned_df['Market Area']
        cleaned_df = cleaned_df.groupby(['Type', 'Fiscal Week'])['Value'].sum().reset_index()
    cleaned_df = cleaned_df.set_index(['Type', 'Fiscal Week']).unstack(['Type'])
    cleaned_df.columns = cleaned_df.columns.get_level_values(0)
    cleaned_df = cleaned_df.reset_index()
    cleaned_df.columns = ['Fiscal Week', 'ARPU', 'ARR', 'Subscribers']
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].astype(str)
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].str[:4] + '-' + cleaned_df['Fiscal Week'].str[4:]
    # remove 53rd week from all years (only 1 occurence)
    cleaned_df[['year', 'week']] = cleaned_df['Fiscal Week'].str.split("-", 1, expand = True)
    cleaned_df = cleaned_df[cleaned_df['week'] != '53']
    del cleaned_df['week']
    del cleaned_df['year']
    cleaned_df['Fiscal Date'] = pd.to_datetime(cleaned_df['Fiscal Week'] + '-1', format = '%Y-%W-%w')
    del cleaned_df['Fiscal Week']
    cleaned_df.index = cleaned_df['Fiscal Date']
    del cleaned_df['Fiscal Date']
    cleaned_df = cleaned_df[:'2023-08-07']
    cleaned_df = cleaned_df[['Subscribers']]
    return cleaned_df

def data_cleaning_traffic_orders_Prophet(df):
    # only Web is represented
    df_copy = df.copy()
    del df_copy['Route to Market - Subscriptions']
    # filter for Geo
    if geo == 'ALL (G)':
        cleaned_df = df_copy.groupby(['Planning DDOM-Traffic_m', 'Fiscal Week'])['Value'].sum().reset_index()
    else:
        cleaned_df = df_copy.loc[df_copy['Market Area'] == geo]
        del cleaned_df['Market Area']
        cleaned_df = cleaned_df.groupby(['Planning DDOM-Traffic_m', 'Fiscal Week'])['Value'].sum().reset_index()
    cleaned_df = cleaned_df.set_index(['Planning DDOM-Traffic_m', 'Fiscal Week']).unstack(['Planning DDOM-Traffic_m'])
    cleaned_df.columns = cleaned_df.columns.get_level_values(0)
    cleaned_df = cleaned_df.reset_index()
    cleaned_df.columns = ['Fiscal Week', 'Traffic', 'Orders']
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].astype(str)
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].str[:4] + '-' + cleaned_df['Fiscal Week'].str[4:]
    # remove 53rd week from all years (only 1 occurence)
    cleaned_df[['year', 'week']] = cleaned_df['Fiscal Week'].str.split("-", 1, expand = True)
    cleaned_df = cleaned_df[cleaned_df['week'] != '53']
    del cleaned_df['week']
    del cleaned_df['year']
    cleaned_df['Fiscal Date'] = pd.to_datetime(cleaned_df['Fiscal Week'] + '-1', format = '%Y-%W-%w')
    del cleaned_df['Fiscal Week']
    return cleaned_df

def data_cleaning_checkout_Prophet(df):
    df_copy = df.copy()
    del df_copy['DDOM Web Funnel Step']
    df_copy = df_copy[df_copy['Unit'] != 0].dropna()
    # filter for Geo
    if geo == 'ALL (G)':
        cleaned_df = df_copy.groupby(['Fiscal Week'])['Unit'].sum().reset_index()
    else:
        cleaned_df = df_copy.loc[df_copy['Market Area'] == geo]
        del cleaned_df['Market Area']
        cleaned_df = cleaned_df.groupby(['Fiscal Week'])['Unit'].sum().reset_index()
    cleaned_df = cleaned_df.set_index(['Fiscal Week'])
    cleaned_df.columns = cleaned_df.columns.get_level_values(0)
    cleaned_df = cleaned_df.reset_index()
    cleaned_df.columns = ['Fiscal Week', 'Checkout']
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].astype(str)
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].str[:4] + '-' + cleaned_df['Fiscal Week'].str[4:]
    cleaned_df[['year', 'week']] = cleaned_df['Fiscal Week'].str.split("-", 1, expand = True)
    cleaned_df = cleaned_df[cleaned_df['week'] != '53']
    del cleaned_df['week']
    del cleaned_df['year']
    cleaned_df['Fiscal Date'] = pd.to_datetime(cleaned_df['Fiscal Week'] + '-1', format = '%Y-%W-%w')
    del cleaned_df['Fiscal Week']
    cleaned_df.index = cleaned_df['Fiscal Date']
    del cleaned_df['Fiscal Date']
    cleaned_df = cleaned_df['2021-12-06':'2023-08-07']
    cleaned_df.reset_index(inplace = True)
    return cleaned_df

def data_cleaning_subscribers_Prophet(df):
    df_copy = df.copy()
    del df_copy['Planning Subscriptions_m']
    # filter for Geo
    if geo == 'ALL (G)':
        cleaned_df = df_copy.groupby(['Type', 'Fiscal Week'])['Value'].sum().reset_index()
    else:
        cleaned_df = df_copy.loc[df_copy['Market Area'] == geo]
        del cleaned_df['Market Area']
        cleaned_df = cleaned_df.groupby(['Type', 'Fiscal Week'])['Value'].sum().reset_index()
    cleaned_df = cleaned_df.set_index(['Type', 'Fiscal Week']).unstack(['Type'])
    cleaned_df.columns = cleaned_df.columns.get_level_values(0)
    cleaned_df = cleaned_df.reset_index()
    cleaned_df.columns = ['Fiscal Week', 'ARPU', 'ARR', 'Subscribers']
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].astype(str)
    cleaned_df['Fiscal Week'] = cleaned_df['Fiscal Week'].str[:4] + '-' + cleaned_df['Fiscal Week'].str[4:]
    # remove 53rd week from all years (only 1 occurence)
    cleaned_df[['year', 'week']] = cleaned_df['Fiscal Week'].str.split("-", 1, expand = True)
    cleaned_df = cleaned_df[cleaned_df['week'] != '53']
    del cleaned_df['week']
    del cleaned_df['year']
    cleaned_df['Fiscal Date'] = pd.to_datetime(cleaned_df['Fiscal Week'] + '-1', format = '%Y-%W-%w')
    del cleaned_df['Fiscal Week']
    cleaned_df.index = cleaned_df['Fiscal Date']
    del cleaned_df['Fiscal Date']
    cleaned_df = cleaned_df[:'2023-08-07']
    cleaned_df = cleaned_df.reset_index()
    cleaned_df = cleaned_df[['Fiscal Date', 'Subscribers']].copy()
    return cleaned_df

def create_traffic_exog_SARIMAX(df):
    df['Traffic Lag 52'] = df['Traffic'].shift(52)
    df['Month Seasonal'] = seasonal_decompose(df['Traffic'], model='additive', period=4).seasonal
    df.dropna(inplace=True)
    df = df['2020-04-06':]
    return df

def create_traffic_exog_Prophet(df):
    df['Traffic Lag 52'] = df['Traffic'].shift(52)
    df.dropna(inplace=True)
    df = df[df['Fiscal Date'] >= '2020-04-06']
    return df

# define a function to calculate the log transformation
def log_transform(data):
    return np.sign(data) * np.log1p(np.abs(data))

# create a function to generate combinations of input list of no.
def pdq_grid(p, d, q):
    pdq = []
    for i in p:
        for j in d:
            for k in q:
                pdq.append([i, j, k])
    return pdq

def PDQm_grid(P, D, Q, m):
    PDQm = []
    for i in P:
        for j in D:
            for k in Q:
                for l in m:
                    PDQm.append([i, j, k, l])
    return PDQm
    

# possible values of the parameters
p = [1, 2, 3]
d = [1]
q = [1, 2, 3]
P = [1]
D = [0]
Q = [1]
m = [13]

# create all combinations of possible values
pdq = pdq_grid(p, d, q)
PDQm = PDQm_grid(P, D, Q, m)

# create a function for semi-grid-searching SARIMA
def SARIMA_grid(train_endog, train_exog, test_endog, test_exog, order, seasonal_order):

    # create an empty list to store values
    model_info = []
    
    # filter away errors & warnings due to failture to converge, LU decomposition errors, etc
    import warnings
    warnings.simplefilter("ignore")
    
    #fit the model
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    from statsmodels.tools.eval_measures import rmse
    
    for i in order:
        for j in seasonal_order:
            model = SARIMAX(endog = train_endog, 
                             exog = train_exog,
                             order=i, seasonal_order=j, 
                             mle_regression = False, 
                             time_varying_regression = True)
            model_fit = model.fit()
            model_test_wrapper = model_fit.get_prediction(start=test_endog.index[0], 
                                                          end=test_endog.index[-1]+timedelta(days=1),
                                                          exog = test_exog)
            model_predict = model_test_wrapper.predicted_mean
            model_predict = np.exp(model_predict)
            
            # calculate evaluation metrics: MAPE, RMSE, AIC & BIC
            MAPE = np.mean(abs((test_endog - model_predict[:-1])/test_endog))
            RMSE = np.sqrt(np.mean((test_endog - model_predict[:-1])**2))
            AIC = model_fit.aic
            BIC = model_fit.bic
            
            # create a list of order, seasonal order & evaluation metrics
            info = [i, j, MAPE, RMSE, AIC, BIC]
            model_info.append(info)

            
    # create a dataframe to store info of all models
    columns = ["order", "seasonal_order", "MAPE", "RMSE", "AIC", "BIC"]
    model_info_output = pd.DataFrame(data=model_info, columns=columns)
    return model_info_output

# functions for month seasonal exogeneous variable (seasonal decomposition)
def roll_to_element(arr, element):
    index = np.where(arr == element)[0]
    rolled_array = np.roll(arr, -index[0] - 1)
    return rolled_array

def tile_with_remainder(arr, length):
    num_reps = length // len(arr)
    tiled_array = np.tile(arr, num_reps + 1)[:length]
    return tiled_array

def Prophet_HyperparameterTuning(train_transformed, test, future):
    param_grid = {  
        'changepoint_prior_scale': [0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1],
        'seasonality_mode': ('multiplicative', 'additive'),
        'changepoint_range': [0.80, 0.90]}

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    MAPEs = []  # Store the MAPEs for each params here

    for params in all_params:
        m = Prophet(**params)
        m.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
        m.fit(train_transformed)
        forecast = m.predict(future)
        forecast['yhat'] = np.exp(forecast['yhat'])
        model_predict = forecast[forecast['ds'] > '2022-12-05']['yhat'].values
        model_MAPE = np.mean(abs((test - model_predict)/test))
        MAPEs.append(model_MAPE)

    tuning_results = pd.DataFrame(all_params)
    tuning_results['MAPE'] = MAPEs
    tuning_results = tuning_results.sort_values(by = 'MAPE')
    tuning_results = tuning_results.reset_index(drop = True)
    return tuning_results

def dampen_prophet(y, fit_df, forecast_df):
    
    predictions = forecast_df.tail(len(forecast_df) - len(fit_df))
    predicted_trend = predictions['trend'].values
    trend_component = fit_df['trend'].values
    
    if 'multiplicative_terms' in forecast_df.columns:
        seasonality_component = fit_df['trend'].values * fit_df['multiplicative_terms'].values
        dampener = TrendDampen(damp_factor=1, damp_style='smooth')
        dampened_trend = dampener.dampen(predicted_trend, y=y, trend_component=trend_component,
                                     seasonality_component=seasonality_component)
        forecasts_damped = predictions['additive_terms'].values + dampened_trend + (dampened_trend * predictions['multiplicative_terms'].values)
    
    else:
        seasonality_component = fit_df['additive_terms'].values
        dampener = TrendDampen(damp_factor=1, damp_style='smooth')
        dampened_trend = dampener.dampen(predicted_trend, y=y, trend_component=trend_component,
                                        seasonality_component=seasonality_component)

        forecasts_damped = predictions['additive_terms'].values + dampened_trend
    return forecasts_damped

# damp_factor = topline_orders['y'][5:].pct_change(periods = 52).mean() ? 

def create_checkout_exog_SARIMAX(df):
    df['Checkout Lag 52'] = df['Checkout'].shift(52)
    df['Month Seasonal'] = seasonal_decompose(df['Checkout'], model='additive', period=4).seasonal
    df['Traffic'] = SARIMAX_traffic_data['Traffic']
    df.dropna(inplace=True)
    df = df['2020-04-06':]
    return df

def create_checkout_exog_Prophet(df):
    df['Checkout Lag 52'] = df['Checkout'].shift(52)
    df['Traffic'] = Prophet_traffic_data[['y']].set_index(df[df['Fiscal Date'] >= '2020-04-06'].index)
    df.dropna(inplace=True)
    df = df[df['Fiscal Date'] >= '2020-04-06']
    return df

def create_orders_exog_SARIMAX(df):
    df['Orders Lag 52'] = df['Orders'].shift(52)
    df['Month Seasonal'] = seasonal_decompose(df['Orders'], model='additive', period=4).seasonal
    df.dropna(inplace=True)
    df = df['2020-04-06':]
    return df

def create_orders_exog_Prophet(df):
    df['Orders Lag 52'] = df['Orders'].shift(52)
    df['Checkout'] = Prophet_checkout_data[['y']].set_index(df[df['Fiscal Date'] >= '2020-04-06'].index)
    df.dropna(inplace=True)
    df = df[df['Fiscal Date'] >= '2020-04-06']
    return df

def create_subscribers_exog_SARIMAX(df):
    df['Subscribers Lag 52'] = df['Subscribers'].shift(52)
    df['Month Seasonal'] = seasonal_decompose(df['Subscribers'], model='additive', period=4).seasonal
    df['Orders'] = SARIMAX_orders_data['Orders']
    df.dropna(inplace=True)
    df = df['2020-04-06':]
    return df

def create_subscribers_exog_Prophet(df):
    df['Subscribers Lag 52'] = df['Subscribers'].shift(52)
    df['Orders'] = Prophet_orders_data[['y']].set_index(df[df['Fiscal Date'] >= '2020-04-06'].index)
    df.dropna(inplace=True)
    df = df[df['Fiscal Date'] >= '2020-04-06']
    return df

# creates backcast for checkout traffic

def checkout_backcast(df):
    i = 0
    np.random.seed(123)
    growth_array = np.random.uniform(0.07, 0.20, 3)
    while i < 3:
        yearly_growth = growth_array[i]
        regressive_decay = 1 - yearly_growth
        np.random.seed(i)
        backcast_indices = pd.date_range(end = df['Fiscal Date'][0], periods = 52, freq = 'W-MON', closed = 'left')
        backcast_values = df[df['Fiscal Date'].isin(backcast_indices.shift(52))]['Checkout'].values * regressive_decay
        random_factors = np.random.uniform(0.95, 1.05, len(backcast_values))
        backcast_values = backcast_values * random_factors
        checkout_backcast = pd.Series(backcast_values, index = backcast_indices).to_frame()
        checkout_backcast.index = checkout_backcast.index.rename('Fiscal Date')
        checkout_backcast = checkout_backcast.rename(columns = {0: 'Checkout'})
        checkout_backcast = checkout_backcast.reset_index()
        df = checkout_backcast.append(df).reset_index(drop = True)
        i = i + 1
    return df

In [9]:
SARIMAX_traffic_orders_data = data_cleaning_traffic_orders_SARIMAX(traffic_orders_data)
Prophet_traffic_orders_data = data_cleaning_traffic_orders_Prophet(traffic_orders_data)

SARIMAX_traffic_data = SARIMAX_traffic_orders_data[['Traffic']]
SARIMAX_orders_data = SARIMAX_traffic_orders_data[['Orders']]
Prophet_traffic_data = Prophet_traffic_orders_data[['Fiscal Date', 'Traffic']]
Prophet_orders_data = Prophet_traffic_orders_data[['Fiscal Date', 'Orders']]

SARIMAX_checkout_data = data_cleaning_checkout_SARIMAX(checkout_traffic_data)
Prophet_checkout_data = data_cleaning_checkout_Prophet(checkout_traffic_data)
Prophet_checkout_data = checkout_backcast(Prophet_checkout_data)
SARIMAX_checkout_data = checkout_backcast(SARIMAX_checkout_data.reset_index()).set_index('Fiscal Date')


SARIMAX_subscribers_data = data_cleaning_subscribers_SARIMAX(subscribers_data)
Prophet_subscribers_data = data_cleaning_subscribers_Prophet(subscribers_data)

In [10]:
# plot surface traffic over time

fig = px.line(x=Prophet_traffic_data['Fiscal Date'], y=Prophet_traffic_data['Traffic'], title="Adobe.com Surface Traffic Over Time", 
              labels={'x': 'Time (in years)', 'y':'No. Actual Visits'})
fig.update_xaxes(rangeslider_visible=True)

pyo.iplot(fig)

In [11]:
# plot checkout traffic over time

fig = px.line(x=Prophet_checkout_data['Fiscal Date'], y=Prophet_checkout_data['Checkout'], title="Adobe.com Checkout Traffic Over Time", 
              labels={'x': 'Time (in years)', 'y':'No. Actual Visits'})
fig.update_xaxes(rangeslider_visible=True)

pyo.iplot(fig)

In [12]:
# plot Adobe.com orders over time

fig = px.line(x=Prophet_orders_data['Fiscal Date'], y=Prophet_orders_data['Orders'], title="Adobe.com Orders Over Time", 
              labels={'x': 'Time (in years)', 'y':'No. Orders'})
fig.update_xaxes(rangeslider_visible=True)

pyo.iplot(fig)

In [13]:
# plot Adobe.com gross new subscribers over time

fig = px.line(x=Prophet_subscribers_data['Fiscal Date'], y=Prophet_subscribers_data['Subscribers'], title="Adobe.com Subscribers Over Time", 
              labels={'x': 'Time (in years)', 'y':'No. Subscribers'})
fig.update_xaxes(rangeslider_visible=True)

pyo.iplot(fig)

# Traffic Forecasting

## SARIMAX 

In [14]:
SARIMAX_traffic_data = create_traffic_exog_SARIMAX(SARIMAX_traffic_data)

# split date is set arbitrarily based on current length of input data into TM1. can be made more dynamic.
split_date = '2022-12-05'

SARIMAX_traffic_data_train = SARIMAX_traffic_data[:split_date]
SARIMAX_traffic_data_test = SARIMAX_traffic_data[split_date:]

# log transform for stationarity
SARIMAX_traffic_data_train_log = SARIMAX_traffic_data_train.copy()
SARIMAX_traffic_data_train_log['Traffic'] = log_transform(SARIMAX_traffic_data_train_log['Traffic'])

SARIMAX_traffic_data_train_log.index.freq = 'W-MON' # Week Start

train_endog = SARIMAX_traffic_data_train_log['Traffic']
train_exog = log_transform(SARIMAX_traffic_data_train_log[['Traffic Lag 52', 'Month Seasonal']])
test_endog = SARIMAX_traffic_data_test['Traffic']
test_exog = log_transform(SARIMAX_traffic_data_test[['Traffic Lag 52', 'Month Seasonal']])

# hyperparameter tuning with custom grid search to find optimal SARIMAX parameters 
model_info = SARIMA_grid(train_endog = train_endog,
                         train_exog = train_exog,
                         test_endog = test_endog,
                         test_exog = test_exog,
                         order = pdq, 
                         seasonal_order = PDQm)

# selects best parameters based on lowest MAPE
model_info = model_info.sort_values(by = 'MAPE')
model_info = model_info.reset_index(drop = True)

order = model_info['order'][0]
seasonal_order = model_info['seasonal_order'][0]

ARIMA_model = sm.tsa.SARIMAX(endog = train_endog, 
                             exog = train_exog,
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_model_fit = ARIMA_model.fit()
ARIMA_train_results = ARIMA_model_fit.fittedvalues

ARIMA_train_results = np.exp(ARIMA_train_results) # reverse transformation

ARIMA_test_predictions_wrapper = ARIMA_model_fit.get_prediction(start=test_endog.index[0], end=test_endog.index[-1]+timedelta(days=1),
                                                 exog = test_exog)

ARIMA_test_predictions = ARIMA_test_predictions_wrapper.predicted_mean

ARIMA_test_predictions = np.exp(ARIMA_test_predictions)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.16092D+00    |proj g|=  5.79476D-01


 This problem is unconstrained.


  ys=-2.229E+00  -gs= 8.517E-01 BFGS update SKIPPED

At iterate    5    f=  1.35631D+00    |proj g|=  1.09090D+01
  ys=-8.467E-01  -gs= 9.501E-01 BFGS update SKIPPED

At iterate   10    f= -1.47139D+00    |proj g|=  8.12521D+01



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     12     55      2     2     0   9.509D+01  -1.471D+00
  F =  -1.4714312999503927     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.16092D+00    |proj g|=  5.79588D-01
  ys=-2.370E+00  -gs= 8.520E-01 BFGS update SKIPPED

At iterate    5    f=  1.31229D+00    |proj g|=  1.13138D+01

At iterate   10    f= -1.48005D+00    |proj g|=  9.02142D+00

           * * *


...


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   11     24    100      3     2     0   6.385D+00  -1.489D+00
  F =  -1.4889405872181543     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.16092D+00    |proj g|=  5.79476D-01
  ys=-2.229E+00  -gs= 8.517E-01 BFGS update SKIPPED


 This problem is unconstrained.



At iterate    5    f=  1.35631D+00    |proj g|=  1.09090D+01
  ys=-8.467E-01  -gs= 9.501E-01 BFGS update SKIPPED

At iterate   10    f= -1.47139D+00    |proj g|=  8.12521D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     12     55      2     2     0   9.509D+01  -1.471D+00
  F =  -1.4714312999503927     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [15]:
# model validation plot for Traffic SARIMAX


# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_train.index[5:], y=SARIMAX_traffic_data_train['Traffic'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_train.index[5:], y=ARIMA_train_results[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_test.index, y=SARIMAX_traffic_data_test['Traffic'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_test.index, y=ARIMA_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Traffic SARIMAX Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

In [16]:
# produce error metrics for Traffic SARIMAX 

SARIMAX_RMSE = np.sqrt(np.mean((ARIMA_test_predictions - SARIMAX_traffic_data_test['Traffic'])**2))

SARIMAX_MAPE = np.mean(abs((SARIMAX_traffic_data_test['Traffic'] - ARIMA_test_predictions[:-1])/SARIMAX_traffic_data_test['Traffic'])) * 100

SARIMAX_TotalError = (sum(ARIMA_test_predictions[:-1]) - sum(SARIMAX_traffic_data_test['Traffic']))
SARIMAX_TotalErrorPercent = abs((SARIMAX_TotalError / sum(SARIMAX_traffic_data_test['Traffic'])) * 100)

metrics = [['SARIMAX', 'Traffic', round(SARIMAX_RMSE), "{:.2%}".format(SARIMAX_MAPE / 100), "{:.2%}".format(SARIMAX_TotalErrorPercent / 100)]]
SARIMAX_TrafficMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

SARIMAX_Traffic_Train_Fit = ARIMA_train_results
SARIMAX_Traffic_Test_Fit = ARIMA_test_predictions

total_forecast_period = 52 + 23
topline_traffic_forecast_set = pd.date_range(start=SARIMAX_traffic_data_test.index[-1], periods=(total_forecast_period - 23), freq='W-MON')
topline_traffic_forecast_set = topline_traffic_forecast_set[1:]

# Create Exogeneous Variables for Forecast Set -- Traffic Lag 52 and Month Seasonal

traffic_lag52_exog_indices = topline_traffic_forecast_set.shift(-52)
traffic_lag52_exog_values = SARIMAX_traffic_data.loc[traffic_lag52_exog_indices]['Traffic'].values
forecast_exog = pd.Series(data = traffic_lag52_exog_values, index = topline_traffic_forecast_set).to_frame().rename(columns={0: 'Traffic Lag 52'})

month_seasonal_values = SARIMAX_traffic_data['Month Seasonal'].unique()
roll_after_index = SARIMAX_traffic_data['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog))

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(SARIMAX_traffic_data['Traffic']), 
                             exog = log_transform(SARIMAX_traffic_data[['Traffic Lag 52', 'Month Seasonal']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.17731D+00    |proj g|=  5.81654D-01
  ys=-2.452E+00  -gs= 8.548E-01 BFGS update SKIPPED

At iterate    5    f=  1.40390D+00    |proj g|=  1.01841D+01
  ys=-3.560E-02  -gs= 8.918E-01 BFGS update SKIPPED

At iterate   10    f= -1.41293D+00    |proj g|=  7.39028D+01

At iterate   15    f= -1.46804D+00    |proj g|=  4.97109D+01
  ys=-6.310E-07  -gs= 2.910E-05 BFGS update SKIPPED

At iterate   20    f= -1.46888D+00    |proj g|=  5.75787D+01

At iterate   25    f= -1.46929D+00    |proj g|=  6.23020D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected 


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [17]:
ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog),
                                                 exog = log_transform(forecast_exog[['Traffic Lag 52', 'Month Seasonal']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean
ARIMA_forecast = np.exp(ARIMA_forecast)

# create initial forecast for one full year out

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data.index[5:], y=SARIMAX_traffic_data['Traffic'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_traffic_forecast_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Traffic SARIMAX Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

In [18]:
# generate a forecast extension to go until FY'24

SARIMAX_Traffic_Full_Fit = ARIMA_fit_results

traffic_forecast = pd.Series(data = ARIMA_forecast.values, index = topline_traffic_forecast_set).to_frame().rename(columns={0: 'Traffic'})
traffic_forecast2 = traffic_forecast.join(forecast_exog)
topline_traffic_with_forecast = SARIMAX_traffic_data.append(traffic_forecast2)
topline_traffic_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 52), freq = 'W-MON')
topline_traffic_forecast_extension_set = topline_traffic_forecast_extension_set[1:]

# Create Exogeneous Variables for Forecast Extension Set -- Traffic Lag 52 and Month Seasonal

traffic_lag52_exog_indices = topline_traffic_forecast_extension_set.shift(-52)
traffic_lag52_exog_values = topline_traffic_with_forecast.loc[traffic_lag52_exog_indices]['Traffic'].values
forecast_exog2 = pd.Series(data = traffic_lag52_exog_values, index = topline_traffic_forecast_extension_set).to_frame().rename(columns={0: 'Traffic Lag 52'})

month_seasonal_values = forecast_exog['Month Seasonal'].unique()
roll_after_index = forecast_exog['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog2['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog2))

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(topline_traffic_with_forecast['Traffic']), 
                             exog = log_transform(topline_traffic_with_forecast[['Traffic Lag 52', 'Month Seasonal']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True);
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog2),
                                                 exog = log_transform(forecast_exog2[['Traffic Lag 52', 'Month Seasonal']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean

ARIMA_forecast = np.exp(ARIMA_forecast)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.18903D+00    |proj g|=  5.84923D-01
  ys=-2.287E+00  -gs= 8.584E-01 BFGS update SKIPPED

At iterate    5    f=  1.39327D+00    |proj g|=  1.04187D+01

At iterate   10    f= -1.51941D+00    |proj g|=  2.30763D+02
  ys=-5.753E-06  -gs= 1.816E-04 BFGS update SKIPPED



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate   15    f= -1.60929D+00    |proj g|=  5.21640D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     16     97      3     2     0   5.216D+00  -1.609D+00
  F =  -1.6092919607199694     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [19]:
# create full forecast for entire FY'24

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_traffic_with_forecast.index[5:], y=topline_traffic_with_forecast['Traffic'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_traffic_with_forecast.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_traffic_forecast_extension_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
#fig.update_layout(title='Adobe.com Traffic SARIMAX Model Forecast -- AMER (G)', xaxis_title='Date', yaxis_title='No. of Actual Visits')
fig.update_layout(title='Traffic SARIMAX Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

In [20]:
# used to export SARIMAX Traffic forecasted values

traffic_forecast_extension = pd.Series(data = ARIMA_forecast.values, index = topline_traffic_forecast_extension_set).to_frame().rename(columns={0: 'Traffic'})
traffic_forecast = traffic_forecast.append(traffic_forecast_extension)
SARIMAX_Traffic_Forecast = traffic_forecast

## Prophet 

In [21]:
Prophet_traffic_data = create_traffic_exog_Prophet(Prophet_traffic_data)
Prophet_traffic_data = Prophet_traffic_data.rename(columns={"Fiscal Date": "ds", "Traffic": "y"})
Prophet_traffic_data_train = Prophet_traffic_data[Prophet_traffic_data['ds'] <= split_date]
Prophet_traffic_data_test = Prophet_traffic_data[Prophet_traffic_data['ds'] > split_date]

# log transform for stationarity
Prophet_traffic_data_train_log = Prophet_traffic_data_train.copy()
Prophet_traffic_data_train_log[['y', 'Traffic Lag 52']] = log_transform(Prophet_traffic_data_train_log[['y', 'Traffic Lag 52']])
model = Prophet()
model.fit(Prophet_traffic_data_train_log)
future = model.make_future_dataframe(periods = len(Prophet_traffic_data_test), freq = 'W-MON')

traffic_forecast_set = pd.date_range(Prophet_traffic_data_train_log['ds'].iloc[-1], periods = len(Prophet_traffic_data_test) + 1, freq = 'W-MON')
traffic_forecast_set = traffic_forecast_set[1:]

# Create Exogeneous Variables -- Traffic Lag 52 

traffic_lag52_exog_indices = traffic_forecast_set.shift(-52)
traffic_lag52_exog_values = Prophet_traffic_data[Prophet_traffic_data['ds'].isin(traffic_lag52_exog_indices)]['y'].values
forecast_exog = pd.Series(data = traffic_lag52_exog_values, index = traffic_forecast_set).to_frame().rename(columns={0: 'Traffic Lag 52'})

forecast_exog = log_transform(forecast_exog)

future['Traffic Lag 52'] = Prophet_traffic_data_train_log['Traffic Lag 52'].append(forecast_exog['Traffic Lag 52']).reset_index(drop = True)

# hyperparameter tuning with custom grid search function for optimal parameter selection
model_info = Prophet_HyperparameterTuning(Prophet_traffic_data_train_log, Prophet_traffic_data_test['y'], future)

15:37:46 - cmdstanpy - INFO - Chain [1] start processing
15:37:47 - cmdstanpy - INFO - Chain [1] done processing
15:37:47 - cmdstanpy - INFO - Chain [1] start processing
15:37:47 - cmdstanpy - INFO - Chain [1] done processing
15:37:47 - cmdstanpy - INFO - Chain [1] start processing
15:37:47 - cmdstanpy - INFO - Chain [1] done processing
15:37:47 - cmdstanpy - INFO - Chain [1] start processing
15:37:47 - cmdstanpy - INFO - Chain [1] done processing
15:37:47 - cmdstanpy - INFO - Chain [1] start processing
15:37:47 - cmdstanpy - INFO - Chain [1] done processing
15:37:47 - cmdstanpy - INFO - Chain [1] start processing
15:37:47 - cmdstanpy - INFO - Chain [1] done processing
15:37:47 - cmdstanpy - INFO - Chain [1] start processing
15:37:47 - cmdstanpy - INFO - Chain [1] done processing
15:37:47 - cmdstanpy - INFO - Chain [1] start processing
15:37:48 - cmdstanpy - INFO - Chain [1] done processing
15:37:48 - cmdstanpy - INFO - Chain [1] start processing
15:37:48 - cmdstanpy - INFO - Chain [1]

In [22]:
del model_info['MAPE']
best_params = model_info.iloc[0].to_dict()
model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Traffic Lag 52')
model.fit(Prophet_traffic_data_train_log)
forecast = model.predict(future)
Prophet_train_predictions = np.exp(forecast[forecast['ds'] <= Prophet_traffic_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])
Prophet_test_predictions = np.exp(forecast[forecast['ds'] > Prophet_traffic_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])

# model validation plot for Traffic Prophet

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_traffic_data_train['ds'][5:], y=Prophet_traffic_data_train['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_traffic_data_train['ds'][5:], y=Prophet_train_predictions[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=Prophet_traffic_data_test['ds'], y=Prophet_traffic_data_test['y'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=Prophet_traffic_data_test['ds'], y=Prophet_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Traffic Prophet Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

15:37:49 - cmdstanpy - INFO - Chain [1] start processing
15:37:49 - cmdstanpy - INFO - Chain [1] done processing


In [23]:
# produce error metrics for Traffic Prophet 

Prophet_RMSE = np.sqrt(np.mean((Prophet_test_predictions.values - Prophet_traffic_data_test['y'].values)**2))

Prophet_MAPE = np.mean(abs((Prophet_traffic_data_test['y'].values - Prophet_test_predictions.values)/Prophet_traffic_data_test['y'].values)) * 100

Prophet_TotalError = (sum(Prophet_test_predictions[:-1].values) - sum(Prophet_traffic_data_test['y'].values))
Prophet_TotalErrorPercent = abs((Prophet_TotalError / sum(Prophet_traffic_data_test['y'].values)) * 100)

metrics = [['Prophet', 'Traffic', round(Prophet_RMSE), "{:.2%}".format(Prophet_MAPE / 100), "{:.2%}".format(Prophet_TotalErrorPercent / 100)]]
Prophet_TrafficMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

Prophet_Traffic_Train_Fit = Prophet_train_predictions
Prophet_Traffic_Test_Fit = Prophet_test_predictions

total_forecast_period = 52 + 22

topline_traffic_forecast_set = pd.date_range(start=Prophet_traffic_data_test['ds'].iloc[-1], periods=(total_forecast_period - 22), freq='W-MON')
topline_traffic_forecast_set = topline_traffic_forecast_set[1:]

Prophet_traffic_data_log = Prophet_traffic_data.copy()
Prophet_traffic_data_log[['y', 'Traffic Lag 52']] = log_transform(Prophet_traffic_data_log[['y', 'Traffic Lag 52']])

model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Traffic Lag 52')
model.fit(Prophet_traffic_data_log)
future = model.make_future_dataframe(periods = (total_forecast_period - 22 - 1), freq = 'W-MON')

traffic_lag52_exog_indices = topline_traffic_forecast_set.shift(-52)
traffic_lag52_exog_values = Prophet_traffic_data[Prophet_traffic_data['ds'].isin(traffic_lag52_exog_indices)]['y'].values

forecast_exog = pd.Series(data = traffic_lag52_exog_values, index = topline_traffic_forecast_set).to_frame().rename(columns={0: 'Traffic Lag 52'})
forecast_exog = log_transform(forecast_exog)
future['Traffic Lag 52'] = Prophet_traffic_data_log['Traffic Lag 52'].append(forecast_exog['Traffic Lag 52']).reset_index(drop = True)

fitted = model.predict()
forecast = model.predict(future)

# dampen the forecast to constrain extrapolating prophet trend 

dampened_forecast = dampen_prophet(Prophet_traffic_data_log['y'].values, fitted, forecast)
full_train = forecast.head(len(forecast) - (total_forecast_period - 22 - 1))
forecast['yhat_damp'] = full_train['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
forecast['yhat'] = np.exp(forecast['yhat'])
forecast['yhat_damp'] = np.exp(forecast['yhat_damp'])
full_train['yhat'] = np.exp(full_train['yhat'])

Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast.tail(total_forecast_period - 22 - 1)['yhat_damp']
Prophet_Traffic_Full_Fit = Prophet_fits_results

# plot one year out Traffic prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_traffic_data['ds'][5:], y=Prophet_traffic_data['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_traffic_data['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_traffic_forecast_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Traffic Prophet Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

Prophet_Traffic_Forecast_WithoutExtension = Prophet_forecast_predictions

15:37:49 - cmdstanpy - INFO - Chain [1] start processing
15:37:49 - cmdstanpy - INFO - Chain [1] done processing


In [24]:
# create forecast extension into FY'24

traffic_forecast = pd.Series(data = Prophet_forecast_predictions.values, index = topline_traffic_forecast_set).to_frame().rename(columns={0: 'y'})
Prophet_Traffic_Full_Fit = Prophet_fits_results
traffic_forecast2 = traffic_forecast.join(forecast_exog)
traffic_forecast2 = traffic_forecast2.reset_index().rename(columns={'index': 'ds'})
traffic_forecast2['Traffic Lag 52'] = np.exp(traffic_forecast2['Traffic Lag 52'])
topline_traffic_with_forecast = Prophet_traffic_data.append(traffic_forecast2)
topline_traffic_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 51), freq = 'W-MON')
topline_traffic_forecast_extension_set = topline_traffic_forecast_extension_set[1:]
topline_traffic_with_forecast_log = topline_traffic_with_forecast.copy()
topline_traffic_with_forecast_log[['y', 'Traffic Lag 52']] = log_transform(topline_traffic_with_forecast_log[['y', 'Traffic Lag 52']])
model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Traffic Lag 52')
model.fit(topline_traffic_with_forecast_log)
future = model.make_future_dataframe(periods = total_forecast_period - 52, freq = 'W-MON')
traffic_lag52_exog_indices = topline_traffic_forecast_extension_set.shift(-52)
traffic_lag52_exog_values = topline_traffic_with_forecast[topline_traffic_with_forecast['ds'].isin(traffic_lag52_exog_indices)]['y'].values
forecast_exog = pd.Series(data = traffic_lag52_exog_values, index = topline_traffic_forecast_extension_set).to_frame().rename(columns={0: 'Traffic Lag 52'})
forecast_exog = log_transform(forecast_exog)
future['Traffic Lag 52'] = topline_traffic_with_forecast_log['Traffic Lag 52'].append(forecast_exog['Traffic Lag 52']).reset_index(drop = True)
fitted = model.predict()
forecast = model.predict(future)
dampened_forecast = np.exp(dampen_prophet(topline_traffic_with_forecast_log['y'].values, fitted, forecast))
forecast['yhat'] = np.exp(forecast['yhat'])
full_train_endpoint = topline_traffic_with_forecast['ds'].iloc[-1]
full_train = forecast[forecast['ds'] <= full_train_endpoint]
forecast['yhat_damp'] = forecast[forecast['ds'] <= full_train_endpoint]['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast[forecast['ds'] > full_train_endpoint]['yhat_damp']

15:37:49 - cmdstanpy - INFO - Chain [1] start processing
15:37:49 - cmdstanpy - INFO - Chain [1] done processing


In [25]:
# plot full FY'24 traffic prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_traffic_with_forecast['ds'][5:], y=topline_traffic_with_forecast['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_traffic_with_forecast['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_traffic_forecast_extension_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
#fig.update_layout(title='Adobe.com Traffic SARIMAX Model Forecast -- AMER (G)', xaxis_title='Date', yaxis_title='No. of Actual Visits')
fig.update_layout(title='Traffic Prophet Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

In [26]:
# used for export of Prophet Traffic forecast

traffic_forecast_extension = pd.Series(data = Prophet_forecast_predictions.values, index = topline_traffic_forecast_extension_set).to_frame().rename(columns={0: 'y'})
traffic_forecast = traffic_forecast.append(traffic_forecast_extension)
Prophet_Traffic_Forecast = traffic_forecast

## Ensemble 

In [27]:
# aggregate results of Prophet and SARIMAX using a mean

Prophet_Traffic_Train_Fit = pd.Series(Prophet_Traffic_Train_Fit.values, SARIMAX_Traffic_Train_Fit.index)
SARIMAX_Traffic_Test_Fit = SARIMAX_Traffic_Test_Fit[1:-1]
Prophet_Traffic_Test_Fit = pd.Series(Prophet_Traffic_Test_Fit.values, SARIMAX_Traffic_Test_Fit.index)
Prophet_Traffic_Full_Fit = pd.Series(Prophet_Traffic_Full_Fit.values, SARIMAX_Traffic_Full_Fit.index)

Ensemble_Traffic_Train_Fit = (Prophet_Traffic_Train_Fit + SARIMAX_Traffic_Train_Fit) / 2
Ensemble_Traffic_Test_Fit = (Prophet_Traffic_Test_Fit + SARIMAX_Traffic_Test_Fit) / 2
Ensemble_Traffic_Full_Fit = (Prophet_Traffic_Full_Fit + SARIMAX_Traffic_Full_Fit) / 2
Ensemble_Traffic_Forecast = (SARIMAX_Traffic_Forecast['Traffic'] + Prophet_Traffic_Forecast['y']) / 2

In [28]:
# model validation plot for Traffic Ensemble

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_train.index[5:], y=SARIMAX_traffic_data_train['Traffic'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_train.index[5:], y=Ensemble_Traffic_Train_Fit.values[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_test.index, y=SARIMAX_traffic_data_test['Traffic'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data_test.index, y=Ensemble_Traffic_Test_Fit.values, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Traffic Ensemble Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

In [29]:
# output full traffic error metrics

Ensemble_RMSE = np.sqrt(np.mean((Ensemble_Traffic_Test_Fit.values - SARIMAX_traffic_data_test['Traffic'][1:].values)**2))
Emsemble_MAPE = np.mean(abs((SARIMAX_traffic_data_test['Traffic'][1:] - Ensemble_Traffic_Test_Fit.values)/SARIMAX_traffic_data_test['Traffic'][1:])) * 100
Ensemble_TotalError = (sum(Ensemble_Traffic_Test_Fit.values) - sum(SARIMAX_traffic_data_test['Traffic'][1:]))
Ensemble_TotalErrorPercent = abs((Ensemble_TotalError / sum(SARIMAX_traffic_data_test['Traffic'][1:])) * 100)

metrics = [['Ensemble', 'Traffic', round(Ensemble_RMSE), "{:.2%}".format(Emsemble_MAPE / 100), "{:.2%}".format(Ensemble_TotalErrorPercent / 100)]]
Ensemble_TrafficMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

TrafficMetrics = SARIMAX_TrafficMetrics.append(Prophet_TrafficMetrics, ignore_index = True)
TrafficMetrics = TrafficMetrics.append(Ensemble_TrafficMetrics, ignore_index = True)
TrafficMetrics.set_index('Model', inplace = True)
TrafficMetrics

,Metric,RMSE,MAPE,Total Error
Model,,,,
SARIMAX,Traffic,2588565,6.65%,4.22%
Prophet,Traffic,1616788,3.83%,4.86%
Ensemble,Traffic,1986678,4.95%,3.20%


In [30]:
# plot full FY'24 traffic ensemble forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data.index[5:], y=SARIMAX_traffic_data['Traffic'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_traffic_data.index[5:], y=Ensemble_Traffic_Full_Fit[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=Ensemble_Traffic_Forecast.index, y=Ensemble_Traffic_Forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
#fig.update_layout(title='Adobe.com Traffic SARIMAX Model Forecast -- AMER (G)', xaxis_title='Date', yaxis_title='No. of Actual Visits')
fig.update_layout(title='Traffic Ensemble Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Actual Visits')

# Show plot
pyo.iplot(fig)

# Checkout Forecasting

## SARIMAX

In [31]:
SARIMAX_checkout_data = create_checkout_exog_SARIMAX(SARIMAX_checkout_data)

# split date is set arbitrarily based on current length of input data into TM1. can be made more dynamic.
split_date = '2022-12-05'

SARIMAX_checkout_data_train = SARIMAX_checkout_data[:split_date]
SARIMAX_checkout_data_test = SARIMAX_checkout_data[split_date:]

# log transform for stationarity
SARIMAX_checkout_data_train_log = SARIMAX_checkout_data_train.copy()
SARIMAX_checkout_data_train_log['Checkout'] = log_transform(SARIMAX_checkout_data_train_log['Checkout'])

SARIMAX_checkout_data_train_log.index.freq = 'W-MON' # Week Start

train_endog = SARIMAX_checkout_data_train_log['Checkout']
train_exog = log_transform(SARIMAX_checkout_data_train_log[['Checkout Lag 52', 'Month Seasonal', 'Traffic']])
test_endog = SARIMAX_checkout_data_test['Checkout']
test_exog = log_transform(SARIMAX_checkout_data_test[['Checkout Lag 52', 'Month Seasonal', 'Traffic']])

# hyperparameter tuning with custom grid search to find optimal SARIMAX parameters 
model_info = SARIMA_grid(train_endog = train_endog,
                         train_exog = train_exog,
                         test_endog = test_endog,
                         test_exog = test_exog,
                         order = pdq, 
                         seasonal_order = PDQm)

# selects best parameters based on lowest MAPE
model_info = model_info.sort_values(by = 'MAPE')
model_info = model_info.reset_index(drop = True)

order = model_info['order'][0]
seasonal_order = model_info['seasonal_order'][0]

ARIMA_model = sm.tsa.SARIMAX(endog = train_endog, 
                             exog = train_exog,
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_model_fit = ARIMA_model.fit()
ARIMA_train_results = ARIMA_model_fit.fittedvalues

ARIMA_train_results = np.exp(ARIMA_train_results) # reverse transformation

ARIMA_test_predictions_wrapper = ARIMA_model_fit.get_prediction(start=test_endog.index[0], end=test_endog.index[-1]+timedelta(days=1),
                                                 exog = test_exog)

ARIMA_test_predictions = ARIMA_test_predictions_wrapper.predicted_mean

ARIMA_test_predictions = np.exp(ARIMA_test_predictions)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.27360D+00    |proj g|=  4.05187D-01


 This problem is unconstrained.



At iterate    5    f=  1.27883D+00    |proj g|=  9.69328D+00

At iterate   10    f= -1.00170D-01    |proj g|=  4.07995D+01

At iterate   15    f= -1.75452D+00    |proj g|=  1.15785D+02

At iterate   20    f= -2.00640D+00    |proj g|=  2.34248D+02

At iterate   25    f= -2.00791D+00    |proj g|=  2.04053D+02

At iterate   30    f= -2.00877D+00    |proj g|=  8.30852D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     30    113      1     0     0   8.309D+01  -2.009D+00
  F =  -2.0087699403817476     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machin


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



At iterate   10    f= -4.88413D-02    |proj g|=  3.76344D+01

At iterate   15    f= -1.78333D+00    |proj g|=  8.98595D+01

At iterate   20    f= -1.86251D+00    |proj g|=  6.10754D+01


...

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.27360D+00    |proj g|=  4.05233D-01

At iterate    5    f=  1.04279D+00    |proj g|=  1.04942D+01

At iterate   10    f= -4.83107D-01    |proj g|=  5.91770D+01

At iterate   15    f= -1.48441D+00    |proj g|=  1.32651D+02

At iterate   20    f= -1.98658D+00    |proj g|=  4.30196D+02



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.


  ys=-7.701E-03  -gs= 2.811E-02 BFGS update SKIPPED



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   10     24    179      4     1     0   3.998D+02  -1.994D+00
  F =  -1.9942781214580505     

ABNORMAL_TERMINATION_IN_LNSRCH                              


In [32]:
# model validation plot for Checkout Traffic SARIMAX

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_train.index[5:], y=SARIMAX_checkout_data_train['Checkout'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_train.index[5:], y=ARIMA_train_results[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_test.index, y=SARIMAX_checkout_data_test['Checkout'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_test.index, y=ARIMA_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Checkout Traffic SARIMAX Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

In [33]:
# produce error metrics for Checkout Traffic SARIMAX

SARIMAX_RMSE = np.sqrt(np.mean((ARIMA_test_predictions - SARIMAX_checkout_data_test['Checkout'])**2))

SARIMAX_MAPE = np.mean(abs((SARIMAX_checkout_data_test['Checkout'] - ARIMA_test_predictions[:-1])/SARIMAX_checkout_data_test['Checkout'])) * 100

SARIMAX_TotalError = (sum(ARIMA_test_predictions[:-1]) - sum(SARIMAX_checkout_data_test['Checkout']))
SARIMAX_TotalErrorPercent = abs((SARIMAX_TotalError / sum(SARIMAX_checkout_data_test['Checkout'])) * 100)

metrics = [['SARIMAX', 'Checkout', round(SARIMAX_RMSE), "{:.2%}".format(SARIMAX_MAPE / 100), "{:.2%}".format(SARIMAX_TotalErrorPercent / 100)]]
SARIMAX_CheckoutMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

SARIMAX_Checkout_Train_Fit = ARIMA_train_results
SARIMAX_Checkout_Test_Fit = ARIMA_test_predictions

total_forecast_period = 52 + 23
topline_checkout_forecast_set = pd.date_range(start=SARIMAX_checkout_data_test.index[-1], periods=(total_forecast_period - 23), freq='W-MON')
topline_checkout_forecast_set = topline_checkout_forecast_set[1:]

# Create Exogeneous Variables for Forecast Set -- Checkout Traffic Lag 52 and Month Seasonal, and Traffic 

checkout_lag52_exog_indices = topline_checkout_forecast_set.shift(-52)
checkout_lag52_exog_values = SARIMAX_checkout_data.loc[checkout_lag52_exog_indices]['Checkout'].values
forecast_exog = pd.Series(data = checkout_lag52_exog_values, index = topline_checkout_forecast_set).to_frame().rename(columns={0: 'Checkout Lag 52'})

month_seasonal_values = SARIMAX_checkout_data['Month Seasonal'].unique()
roll_after_index = SARIMAX_checkout_data['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog))
forecast_exog['Traffic'] = SARIMAX_Traffic_Forecast[SARIMAX_Traffic_Forecast.index < traffic_forecast_extension.index[0]]

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(SARIMAX_checkout_data['Checkout']), 
                             exog = log_transform(SARIMAX_checkout_data[['Checkout Lag 52', 'Month Seasonal', 'Traffic']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.29626D+00    |proj g|=  4.06531D-01

At iterate    5    f=  1.28570D+00    |proj g|=  1.15186D+01

At iterate   10    f= -1.43019D+00    |proj g|=  1.12122D+02

At iterate   15    f= -1.56298D+00    |proj g|=  4.78395D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   10     15     58      1     0     0   4.784D+01  -1.563D+00
  F =  -1.5629838443568647     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [34]:
ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog),
                                                 exog = log_transform(forecast_exog[['Checkout Lag 52', 'Month Seasonal', 'Traffic']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean
ARIMA_forecast = np.exp(ARIMA_forecast)

# create initial forecast for one full year out

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data.index[5:], y=SARIMAX_checkout_data['Checkout'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_checkout_forecast_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Checkout Traffic SARIMAX Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

In [35]:
# generate a forecast extension to go until FY'24

SARIMAX_Checkout_Full_Fit = ARIMA_fit_results

checkout_forecast = pd.Series(data = ARIMA_forecast.values, index = topline_checkout_forecast_set).to_frame().rename(columns={0: 'Checkout'})
checkout_forecast2 = checkout_forecast.join(forecast_exog)
topline_checkout_with_forecast = SARIMAX_checkout_data.append(checkout_forecast2)
topline_checkout_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 52), freq = 'W-MON')
topline_checkout_forecast_extension_set = topline_checkout_forecast_extension_set[1:]

# Create Exogeneous Variables for Forecast Extension Set -- Checkout Traffic Lag 52, Month Seasonal, and Traffic

checkout_lag52_exog_indices = topline_checkout_forecast_extension_set.shift(-52)
checkout_lag52_exog_values = topline_checkout_with_forecast.loc[checkout_lag52_exog_indices]['Checkout'].values
forecast_exog2 = pd.Series(data = checkout_lag52_exog_values, index = topline_checkout_forecast_extension_set).to_frame().rename(columns={0: 'Checkout Lag 52'})

month_seasonal_values = forecast_exog['Month Seasonal'].unique()
roll_after_index = forecast_exog['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog2['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog2))
forecast_exog2['Traffic'] = SARIMAX_Traffic_Forecast[SARIMAX_Traffic_Forecast.index >= traffic_forecast_extension.index[0]]

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(topline_checkout_with_forecast['Checkout']), 
                             exog = log_transform(topline_checkout_with_forecast[['Checkout Lag 52', 'Month Seasonal', 'Traffic']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True);
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog2),
                                                 exog = log_transform(forecast_exog2[['Checkout Lag 52', 'Month Seasonal', 'Traffic']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean

ARIMA_forecast = np.exp(ARIMA_forecast)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.31671D+00    |proj g|=  4.08384D-01

At iterate    5    f=  3.09730D-01    |proj g|=  2.37809D+01
  ys=-1.595E+00  -gs= 9.764E-01 BFGS update SKIPPED

At iterate   10    f= -1.65039D+00    |proj g|=  8.71261D+01

At iterate   15    f= -1.71265D+00    |proj g|=  1.39835D+01

At iterate   20    f= -1.71309D+00    |proj g|=  4.30807D+01
  ys=-6.097E-06  -gs= 1.554E-05 BFGS update SKIPPED
  ys=-1.662E-06  -gs= 2.818E-06 BFGS update SKIPPED



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   10     25    139      2     3     0   4.859D+01  -1.713D+00
  F =  -1.7133788295230639     

ABNORMAL_TERMINATION_IN_LNSRCH                              



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [36]:
# create full forecast for entire FY'24

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_checkout_with_forecast.index[5:], y=topline_checkout_with_forecast['Checkout'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_checkout_with_forecast.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_checkout_forecast_extension_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Checkout Traffic SARIMAX Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

In [37]:
# used to export SARIMAX Checkout Traffic forecasted values

checkout_forecast_extension = pd.Series(data = ARIMA_forecast.values, index = topline_checkout_forecast_extension_set).to_frame().rename(columns={0: 'Checkout'})
checkout_forecast = checkout_forecast.append(checkout_forecast_extension)
SARIMAX_Checkout_Forecast = checkout_forecast

## Prophet

In [38]:
Prophet_checkout_data = create_checkout_exog_Prophet(Prophet_checkout_data)
Prophet_checkout_data = Prophet_checkout_data.rename(columns={"Fiscal Date": "ds", "Checkout": "y"})
Prophet_checkout_data_train = Prophet_checkout_data[Prophet_checkout_data['ds'] <= split_date]
Prophet_checkout_data_test = Prophet_checkout_data[Prophet_checkout_data['ds'] > split_date]

# log transform for stationarity
Prophet_checkout_data_train_log = Prophet_checkout_data_train.copy()
Prophet_checkout_data_train_log[['y', 'Checkout Lag 52', 'Traffic']] = log_transform(Prophet_checkout_data_train_log[['y', 'Checkout Lag 52', 'Traffic']])

model = Prophet()
model.fit(Prophet_checkout_data_train_log)
future = model.make_future_dataframe(periods = len(Prophet_checkout_data_test), freq = 'W-MON')
checkout_forecast_set = pd.date_range(Prophet_checkout_data_train_log['ds'].iloc[-1], periods = len(Prophet_checkout_data_test) + 1, freq = 'W-MON')
checkout_forecast_set = checkout_forecast_set[1:]

# Create Exogeneous Variables -- Checkout Traffic Lag 52 and Traffic

checkout_lag52_exog_indices = checkout_forecast_set.shift(-52)
checkout_lag52_exog_values = Prophet_checkout_data[Prophet_checkout_data['ds'].isin(checkout_lag52_exog_indices)]['y'].values

forecast_exog = pd.Series(data = checkout_lag52_exog_values, index = checkout_forecast_set).to_frame().rename(columns={0: 'Checkout Lag 52'})
forecast_exog['Traffic'] = Prophet_traffic_data_test['y'].values
forecast_exog = log_transform(forecast_exog)

future['Checkout Lag 52'] = Prophet_checkout_data_train_log['Checkout Lag 52'].append(forecast_exog['Checkout Lag 52']).reset_index(drop = True)
future['Traffic'] = Prophet_traffic_data_train_log['y'].append(forecast_exog['Traffic']).reset_index(drop = True)

# hyperparameter tuning with custom grid search function for optimal parameter selection
model_info = Prophet_HyperparameterTuning(Prophet_checkout_data_train_log, Prophet_checkout_data_test['y'], future)

15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing
15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing
15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing
15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing
15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing
15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing
15:38:24 - cmdstanpy - INFO - Chain [1] start processing
15:38:24 - cmdstanpy - INFO - Chain [1] done processing
15:38:25 - cmdstanpy - INFO - Chain [1] start processing
15:38:25 - cmdstanpy - INFO - Chain [1] done processing
15:38:25 - cmdstanpy - INFO - Chain [1] start processing
15:38:25 - cmdstanpy - INFO - Chain [1]

In [39]:
del model_info['MAPE']
best_params = model_info.iloc[0].to_dict()

model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Checkout Lag 52')
model.add_regressor('Traffic')
model.fit(Prophet_checkout_data_train_log)
forecast = model.predict(future)
Prophet_train_predictions = np.exp(forecast[forecast['ds'] <= Prophet_checkout_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])
Prophet_test_predictions = np.exp(forecast[forecast['ds'] > Prophet_checkout_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])

# model validation plot for Checkout Traffic Prophet

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_checkout_data_train['ds'][5:], y=Prophet_checkout_data_train['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_checkout_data_train['ds'][5:], y=Prophet_train_predictions[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=Prophet_checkout_data_test['ds'], y=Prophet_checkout_data_test['y'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=Prophet_checkout_data_test['ds'], y=Prophet_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Checkout Prophet Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

15:38:25 - cmdstanpy - INFO - Chain [1] start processing
15:38:25 - cmdstanpy - INFO - Chain [1] done processing


In [40]:
# produce error metrics for Checkout Traffic Prophet 

Prophet_RMSE = np.sqrt(np.mean((Prophet_test_predictions.values - Prophet_checkout_data_test['y'].values)**2))

Prophet_MAPE = np.mean(abs((Prophet_checkout_data_test['y'].values - Prophet_test_predictions.values)/Prophet_checkout_data_test['y'].values)) * 100

Prophet_TotalError = (sum(Prophet_test_predictions[:-1].values) - sum(Prophet_checkout_data_test['y'].values))
Prophet_TotalErrorPercent = abs((Prophet_TotalError / sum(Prophet_checkout_data_test['y'].values)) * 100)

metrics = [['Prophet', 'Checkout', round(Prophet_RMSE), "{:.2%}".format(Prophet_MAPE / 100), "{:.2%}".format(Prophet_TotalErrorPercent / 100)]]
Prophet_CheckoutMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

Prophet_Checkout_Train_Fit = Prophet_train_predictions
Prophet_Checkout_Test_Fit = Prophet_test_predictions

total_forecast_period = 52 + 22

topline_checkout_forecast_set = pd.date_range(start=Prophet_checkout_data_test['ds'].iloc[-1], periods=(total_forecast_period - 22), freq='W-MON')
topline_checkout_forecast_set = topline_checkout_forecast_set[1:]

Prophet_checkout_data_log = Prophet_checkout_data.copy()
Prophet_checkout_data_log[['y', 'Checkout Lag 52', 'Traffic']] = log_transform(Prophet_checkout_data_log[['y', 'Checkout Lag 52', 'Traffic']])

model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Checkout Lag 52')
model.add_regressor('Traffic')
model.fit(Prophet_checkout_data_log)
future = model.make_future_dataframe(periods = (total_forecast_period - 22 - 1), freq = 'W-MON')

checkout_lag52_exog_indices = topline_checkout_forecast_set.shift(-52)
checkout_lag52_exog_values = Prophet_checkout_data[Prophet_checkout_data['ds'].isin(checkout_lag52_exog_indices)]['y'].values

forecast_exog = pd.Series(data = checkout_lag52_exog_values, index = topline_checkout_forecast_set).to_frame().rename(columns={0: 'Checkout Lag 52'})
forecast_exog['Traffic'] = Prophet_Traffic_Forecast_WithoutExtension.values
forecast_exog = log_transform(forecast_exog)
future['Checkout Lag 52'] = Prophet_checkout_data_log['Checkout Lag 52'].append(forecast_exog['Checkout Lag 52']).reset_index(drop = True)
future['Traffic'] = Prophet_traffic_data_log['y'].append(forecast_exog['Traffic']).reset_index(drop = True)

fitted = model.predict()
forecast = model.predict(future)

# dampen the forecast to constrain extrapolating prophet trend 

dampened_forecast = dampen_prophet(Prophet_checkout_data_log['y'].values, fitted, forecast)
full_train = forecast.head(len(forecast) - (total_forecast_period - 22 - 1))
forecast['yhat_damp'] = full_train['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
forecast['yhat'] = np.exp(forecast['yhat'])
forecast['yhat_damp'] = np.exp(forecast['yhat_damp'])
full_train['yhat'] = np.exp(full_train['yhat'])

Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast.tail(total_forecast_period - 22 - 1)['yhat_damp']
Prophet_Subscribers_Full_Fit = Prophet_fits_results

# plot one year out Checkout Traffic prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_checkout_data['ds'][5:], y=Prophet_checkout_data['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_checkout_data['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_checkout_forecast_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Checkout Traffic Prophet Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

Prophet_Checkout_Forecast_WithoutExtension = Prophet_forecast_predictions

15:38:26 - cmdstanpy - INFO - Chain [1] start processing
15:38:26 - cmdstanpy - INFO - Chain [1] done processing


In [41]:
# create forecast extension into FY'24

checkout_forecast = pd.Series(data = Prophet_forecast_predictions.values, index = topline_checkout_forecast_set).to_frame().rename(columns={0: 'y'})
Prophet_Checkout_Full_Fit = Prophet_fits_results
checkout_forecast2 = checkout_forecast.join(forecast_exog)
checkout_forecast2 = checkout_forecast2.reset_index().rename(columns={'index': 'ds'})
checkout_forecast2['Checkout Lag 52'] = np.exp(checkout_forecast2['Checkout Lag 52'])
topline_checkout_with_forecast = Prophet_checkout_data.append(checkout_forecast2)

topline_checkout_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 51), freq = 'W-MON')
topline_checkout_forecast_extension_set = topline_checkout_forecast_extension_set[1:]

topline_checkout_with_forecast_log = topline_checkout_with_forecast.copy()
topline_checkout_with_forecast_log[['y', 'Checkout Lag 52', 'Traffic']] = log_transform(topline_checkout_with_forecast_log[['y', 'Checkout Lag 52', 'Traffic']])
model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Checkout Lag 52')
model.add_regressor('Traffic')

model.fit(topline_checkout_with_forecast_log)
future = model.make_future_dataframe(periods = total_forecast_period - 52, freq = 'W-MON')
checkout_lag52_exog_indices = topline_checkout_forecast_extension_set.shift(-52)
checkout_lag52_exog_values = topline_checkout_with_forecast[topline_checkout_with_forecast['ds'].isin(checkout_lag52_exog_indices)]['y'].values
forecast_exog = pd.Series(data = checkout_lag52_exog_values, index = topline_checkout_forecast_extension_set).to_frame().rename(columns={0: 'Checkout Lag 52'})

forecast_exog['Traffic'] = traffic_forecast_extension.values

forecast_exog = log_transform(forecast_exog)

future['Checkout Lag 52'] = topline_checkout_with_forecast_log['Checkout Lag 52'].append(forecast_exog['Checkout Lag 52']).reset_index(drop = True)

future['Traffic'] = topline_traffic_with_forecast_log['y'].append(forecast_exog['Traffic']).reset_index(drop = True)


fitted = model.predict()
forecast = model.predict(future)
dampened_forecast = np.exp(dampen_prophet(topline_checkout_with_forecast_log['y'].values, fitted, forecast))
forecast['yhat'] = np.exp(forecast['yhat'])
full_train_endpoint = topline_checkout_with_forecast['ds'].iloc[-1]
full_train = forecast[forecast['ds'] <= full_train_endpoint]
forecast['yhat_damp'] = forecast[forecast['ds'] <= full_train_endpoint]['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast[forecast['ds'] > full_train_endpoint]['yhat_damp']

15:38:26 - cmdstanpy - INFO - Chain [1] start processing
15:38:26 - cmdstanpy - INFO - Chain [1] done processing


In [42]:
# plot full FY'24 checkout traffic prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_checkout_with_forecast['ds'][5:], y=topline_checkout_with_forecast['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_checkout_with_forecast['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_checkout_forecast_extension_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Checkout Traffic Prophet Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

In [43]:
# used for export of Prophet Checkout Traffic forecast

checkout_forecast_extension = pd.Series(data = Prophet_forecast_predictions.values, index = topline_checkout_forecast_extension_set).to_frame().rename(columns={0: 'y'})
checkout_forecast = checkout_forecast.append(checkout_forecast_extension)
Prophet_Checkout_Forecast = checkout_forecast

## Ensemble

In [44]:
# aggregate results of Prophet and SARIMAX using a mean

Prophet_Checkout_Train_Fit = pd.Series(Prophet_Checkout_Train_Fit.values, SARIMAX_Checkout_Train_Fit.index)
SARIMAX_Checkout_Test_Fit = SARIMAX_Checkout_Test_Fit[1:-1]
Prophet_Checkout_Test_Fit = pd.Series(Prophet_Checkout_Test_Fit.values, SARIMAX_Checkout_Test_Fit.index)
Prophet_Checkout_Full_Fit = pd.Series(Prophet_Checkout_Full_Fit.values, SARIMAX_Checkout_Full_Fit.index)

Ensemble_Checkout_Train_Fit = (Prophet_Checkout_Train_Fit + SARIMAX_Checkout_Train_Fit) / 2
Ensemble_Checkout_Test_Fit = (Prophet_Checkout_Test_Fit + SARIMAX_Checkout_Test_Fit) / 2
Ensemble_Checkout_Full_Fit = (Prophet_Checkout_Full_Fit + SARIMAX_Checkout_Full_Fit) / 2
Ensemble_Checkout_Forecast = (SARIMAX_Checkout_Forecast['Checkout'] + Prophet_Checkout_Forecast['y']) / 2

In [45]:
# model validation plot for Checkout Traffic Ensemble

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_train.index[5:], y=SARIMAX_checkout_data_train['Checkout'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_train.index[5:], y=Ensemble_Checkout_Train_Fit.values[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_test.index, y=SARIMAX_checkout_data_test['Checkout'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data_test.index, y=Ensemble_Checkout_Test_Fit.values, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Checkout Traffic Ensemble Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

In [46]:
# output full checkout traffic error metrics

Ensemble_RMSE = np.sqrt(np.mean((Ensemble_Checkout_Test_Fit.values - SARIMAX_checkout_data_test['Checkout'][1:].values)**2))
Emsemble_MAPE = np.mean(abs((SARIMAX_checkout_data_test['Checkout'][1:] - Ensemble_Checkout_Test_Fit.values)/SARIMAX_checkout_data_test['Checkout'][1:])) * 100
Ensemble_TotalError = (sum(Ensemble_Checkout_Test_Fit.values) - sum(SARIMAX_checkout_data_test['Checkout'][1:]))
Ensemble_TotalErrorPercent = abs((Ensemble_TotalError / sum(SARIMAX_checkout_data_test['Checkout'][1:])) * 100)

metrics = [['Ensemble', 'Checkout', round(Ensemble_RMSE), "{:.2%}".format(Emsemble_MAPE / 100), "{:.2%}".format(Ensemble_TotalErrorPercent / 100)]]
Ensemble_CheckoutMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

CheckoutMetrics = SARIMAX_CheckoutMetrics.append(Prophet_CheckoutMetrics, ignore_index = True)
CheckoutMetrics = CheckoutMetrics.append(Ensemble_CheckoutMetrics, ignore_index = True)
CheckoutMetrics.set_index('Model', inplace = True)
CheckoutMetrics

,Metric,RMSE,MAPE,Total Error
Model,,,,
SARIMAX,Checkout,170347,5.01%,0.27%
Prophet,Checkout,227163,6.65%,1.70%
Ensemble,Checkout,166602,5.02%,2.47%


In [47]:
# plot full FY'24 checkout traffic ensemble forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data.index[5:], y=SARIMAX_checkout_data['Checkout'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_checkout_data.index[5:], y=Ensemble_Checkout_Full_Fit[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=Ensemble_Checkout_Forecast.index, y=Ensemble_Checkout_Forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
#fig.update_layout(title='Adobe.com Traffic SARIMAX Model Forecast -- AMER (G)', xaxis_title='Date', yaxis_title='No. of Actual Visits')
fig.update_layout(title='Checkout Traffic Ensemble Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Checkout Visits')

# Show plot
pyo.iplot(fig)

# Orders Forecasting

## SARIMAX

In [48]:
SARIMAX_orders_data = create_orders_exog_SARIMAX(SARIMAX_orders_data)

# split date is set arbitrarily based on current length of input data into TM1. can be made more dynamic.
split_date = '2022-12-05'

SARIMAX_orders_data_train = SARIMAX_orders_data[:split_date]
SARIMAX_orders_data_test = SARIMAX_orders_data[split_date:]

# log transform for stationarity
SARIMAX_orders_data_train_log = SARIMAX_orders_data_train.copy()
SARIMAX_orders_data_train_log['Orders'] = log_transform(SARIMAX_orders_data_train_log['Orders'])

SARIMAX_orders_data_train_log.index.freq = 'W-MON' # Week Start

train_endog = SARIMAX_orders_data_train_log['Orders']
train_exog = log_transform(SARIMAX_orders_data_train_log[['Orders Lag 52', 'Month Seasonal']])
test_endog = SARIMAX_orders_data_test['Orders']
test_exog = log_transform(SARIMAX_orders_data_test[['Orders Lag 52', 'Month Seasonal']])

# hyperparameter tuning with custom grid search to find optimal SARIMAX parameters 
model_info = SARIMA_grid(train_endog = train_endog,
                         train_exog = train_exog,
                         test_endog = test_endog,
                         test_exog = test_exog,
                         order = pdq, 
                         seasonal_order = PDQm)

# selects best parameters based on lowest MAPE
model_info = model_info.sort_values(by = 'MAPE')
model_info = model_info.reset_index(drop = True)

order = model_info['order'][0]
seasonal_order = model_info['seasonal_order'][0]

ARIMA_model = sm.tsa.SARIMAX(endog = train_endog, 
                             exog = train_exog,
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_model_fit = ARIMA_model.fit()
ARIMA_train_results = ARIMA_model_fit.fittedvalues

ARIMA_train_results = np.exp(ARIMA_train_results) # reverse transformation

ARIMA_test_predictions_wrapper = ARIMA_model_fit.get_prediction(start=test_endog.index[0], end=test_endog.index[-1]+timedelta(days=1),
                                                 exog = test_exog)

ARIMA_test_predictions = ARIMA_test_predictions_wrapper.predicted_mean

ARIMA_test_predictions = np.exp(ARIMA_test_predictions)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.86107D+00    |proj g|=  6.19994D-01

At iterate    5    f= -6.41842D-01    |proj g|=  3.35279D+01

At iterate   10    f= -8.95185D-01    |proj g|=  2.99543D+00

At iterate   15    f= -8.95224D-01    |proj g|=  4.96557D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     16     52      1     0     0   4.966D-01  -8.952D-01
  F = -0.89522393326205807     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING 

 This problem is unconstrained.


  ys=-8.507E+00  -gs= 9.613E-01 BFGS update SKIPPED

At iterate    5    f= -8.67506D-01    |proj g|=  1.40476D+01

At iterate   10    f= -8.75914D-01    |proj g|=  2.92891D+00

At iterate   15    f= -9.15180D-01    |proj g|=  4.43278D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     17     52      1     1     0   8.066D+00  -9.159D-01
  F = -0.91585321990118995     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0  

 This problem is unconstrained.


...


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   10     17     75      1     2     0   5.850D-01  -8.180D-01
  F = -0.81797280290063734     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.86106D+00    |proj g|=  6.19665D-01

At iterate    5    f= -8.07372D-01    |proj g|=  2.44794D+01

At iterate   10    f= -8.62143D-01    |proj g|=  6.97567D+00

At iterate   15    f= -8.78809D-01    |proj g|=  2.35088D+01

At iter


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   11     28    102      2     0     0   3.668D-01  -8.893D-01
  F = -0.88930914800419170     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.86106D+00    |proj g|=  6.19766D-01

At iterate    5    f= -8.43380D-01    |proj g|=  2.40952D+01

At iterate   10    f= -8.57668D-01    |proj g|=  1.10717D+00

At iterate   15    f= -8.73958D-01    |proj g|=  1.98937D+01

At iter


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [49]:
# model validation plot for Orders SARIMAX

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_train.index[5:], y=SARIMAX_orders_data_train['Orders'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_train.index[5:], y=ARIMA_train_results[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_test.index, y=SARIMAX_orders_data_test['Orders'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_test.index, y=ARIMA_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Orders SARIMAX Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

In [50]:
# produce error metrics for Orders SARIMAX 

SARIMAX_RMSE = np.sqrt(np.mean((ARIMA_test_predictions - SARIMAX_orders_data_test['Orders'])**2))

SARIMAX_MAPE = np.mean(abs((SARIMAX_orders_data_test['Orders'] - ARIMA_test_predictions[:-1])/SARIMAX_orders_data_test['Orders'])) * 100

SARIMAX_TotalError = (sum(ARIMA_test_predictions[:-1]) - sum(SARIMAX_orders_data_test['Orders']))
SARIMAX_TotalErrorPercent = abs((SARIMAX_TotalError / sum(SARIMAX_orders_data_test['Orders'])) * 100)

metrics = [['SARIMAX', 'Orders', round(SARIMAX_RMSE), "{:.2%}".format(SARIMAX_MAPE / 100), "{:.2%}".format(SARIMAX_TotalErrorPercent / 100)]]
SARIMAX_OrdersMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

SARIMAX_Orders_Train_Fit = ARIMA_train_results
SARIMAX_Orders_Test_Fit = ARIMA_test_predictions

total_forecast_period = 52 + 23
topline_orders_forecast_set = pd.date_range(start=SARIMAX_orders_data_test.index[-1], periods=(total_forecast_period - 23), freq='W-MON')
topline_orders_forecast_set = topline_orders_forecast_set[1:]

# Create Exogeneous Variables for Forecast Set -- Orders Lag 52 and Month Seasonal

orders_lag52_exog_indices = topline_orders_forecast_set.shift(-52)
orders_lag52_exog_values = SARIMAX_orders_data.loc[orders_lag52_exog_indices]['Orders'].values
forecast_exog = pd.Series(data = orders_lag52_exog_values, index = topline_orders_forecast_set).to_frame().rename(columns={0: 'Orders Lag 52'})

month_seasonal_values = SARIMAX_orders_data['Month Seasonal'].unique()
roll_after_index = SARIMAX_orders_data['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog))

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(SARIMAX_orders_data['Orders']), 
                             exog = log_transform(SARIMAX_orders_data[['Orders Lag 52', 'Month Seasonal']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.87424D+00    |proj g|=  6.23296D-01


 This problem is unconstrained.


  ys=-1.024E+01  -gs= 9.746E-01 BFGS update SKIPPED

At iterate    5    f= -8.76703D-01    |proj g|=  1.54482D+01

At iterate   10    f= -9.03211D-01    |proj g|=  3.17200D+00

At iterate   15    f= -9.50077D-01    |proj g|=  1.91746D+01

At iterate   20    f= -9.57499D-01    |proj g|=  2.33570D+00
  ys=-1.507E-06  -gs= 3.284E-06 BFGS update SKIPPED

At iterate   25    f= -9.57549D-01    |proj g|=  1.14193D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   10     25    117      1     2     0   1.142D+01  -9.575D-01
  F = -0.95754941572836827     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [51]:
ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog),
                                                 exog = log_transform(forecast_exog[['Orders Lag 52', 'Month Seasonal']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean
ARIMA_forecast = np.exp(ARIMA_forecast)

# create initial forecast for one full year out

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data.index[5:], y=SARIMAX_orders_data['Orders'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_orders_forecast_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Orders SARIMAX Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

In [52]:
# generate a forecast extension to go until FY'24

SARIMAX_Orders_Full_Fit = ARIMA_fit_results

orders_forecast = pd.Series(data = ARIMA_forecast.values, index = topline_orders_forecast_set).to_frame().rename(columns={0: 'Orders'})
orders_forecast2 = orders_forecast.join(forecast_exog)
topline_orders_with_forecast = SARIMAX_orders_data.append(orders_forecast2)
topline_orders_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 52), freq = 'W-MON')
topline_orders_forecast_extension_set = topline_orders_forecast_extension_set[1:]

# Create Exogeneous Variables for Forecast Extension Set -- Orders Lag 52 and Month Seasonal

orders_lag52_exog_indices = topline_orders_forecast_extension_set.shift(-52)
orders_lag52_exog_values = topline_orders_with_forecast.loc[orders_lag52_exog_indices]['Orders'].values
forecast_exog2 = pd.Series(data = orders_lag52_exog_values, index = topline_orders_forecast_extension_set).to_frame().rename(columns={0: 'Orders Lag 52'})

month_seasonal_values = forecast_exog['Month Seasonal'].unique()
roll_after_index = forecast_exog['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog2['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog2))

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(topline_orders_with_forecast['Orders']), 
                             exog = log_transform(topline_orders_with_forecast[['Orders Lag 52', 'Month Seasonal']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True);
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog2),
                                                 exog = log_transform(forecast_exog2[['Orders Lag 52', 'Month Seasonal']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean

ARIMA_forecast = np.exp(ARIMA_forecast)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.88731D+00    |proj g|=  6.26377D-01


 This problem is unconstrained.


  ys=-2.424E+00  -gs= 9.770E-01 BFGS update SKIPPED

At iterate    5    f= -1.03280D+00    |proj g|=  1.01047D+01

At iterate   10    f= -1.06173D+00    |proj g|=  1.13223D+02

At iterate   15    f= -1.09556D+00    |proj g|=  1.36635D+01

At iterate   20    f= -1.09637D+00    |proj g|=  3.03946D+00



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



At iterate   25    f= -1.09640D+00    |proj g|=  5.29262D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   10     25    108      2     1     0   5.293D-01  -1.096D+00
  F =  -1.0964030731820527     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [53]:
# create full forecast for entire FY'24

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_orders_with_forecast.index[5:], y=topline_orders_with_forecast['Orders'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_orders_with_forecast.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_orders_forecast_extension_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Orders SARIMAX Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

In [54]:
# used to export SARIMAX Orders forecasted values

orders_forecast_extension = pd.Series(data = ARIMA_forecast.values, index = topline_orders_forecast_extension_set).to_frame().rename(columns={0: 'Orders'})
orders_forecast = orders_forecast.append(orders_forecast_extension)
SARIMAX_Orders_Forecast = orders_forecast

## Prophet

In [55]:
Prophet_orders_data = create_orders_exog_Prophet(Prophet_orders_data)
Prophet_orders_data = Prophet_orders_data.rename(columns={"Fiscal Date": "ds", "Orders": "y"})
Prophet_orders_data_train = Prophet_orders_data[Prophet_orders_data['ds'] <= split_date]
Prophet_orders_data_test = Prophet_orders_data[Prophet_orders_data['ds'] > split_date]
Prophet_orders_data_train_log = Prophet_orders_data_train.copy()

# log transform for stationarity
Prophet_orders_data_train_log[['y', 'Orders Lag 52', 'Checkout']] = log_transform(Prophet_orders_data_train_log[['y', 'Orders Lag 52', 'Checkout']])
model = Prophet()
model.fit(Prophet_orders_data_train_log)
future = model.make_future_dataframe(periods = len(Prophet_orders_data_test), freq = 'W-MON')
orders_forecast_set = pd.date_range(Prophet_orders_data_train_log['ds'].iloc[-1], periods = len(Prophet_orders_data_test) + 1, freq = 'W-MON')
orders_forecast_set = orders_forecast_set[1:]

# Create Exogeneous Variables -- Orders Lag 52 and Checkout Traffic

orders_lag52_exog_indices = orders_forecast_set.shift(-52)
orders_lag52_exog_values = Prophet_orders_data[Prophet_orders_data['ds'].isin(orders_lag52_exog_indices)]['y'].values

forecast_exog = pd.Series(data = orders_lag52_exog_values, index = orders_forecast_set).to_frame().rename(columns={0: 'Orders Lag 52'})
forecast_exog['Checkout'] = Prophet_checkout_data_test['y'].values
forecast_exog = log_transform(forecast_exog)

future['Orders Lag 52'] = Prophet_orders_data_train_log['Orders Lag 52'].append(forecast_exog['Orders Lag 52']).reset_index(drop = True)
future['Checkout'] = Prophet_checkout_data_train_log['y'].append(forecast_exog['Checkout']).reset_index(drop = True)

# hyperparameter tuning with custom grid search function for optimal parameter selection
model_info = Prophet_HyperparameterTuning(Prophet_orders_data_train_log, Prophet_orders_data_test['y'], future)

15:38:43 - cmdstanpy - INFO - Chain [1] start processing
15:38:43 - cmdstanpy - INFO - Chain [1] done processing
15:38:43 - cmdstanpy - INFO - Chain [1] start processing
15:38:43 - cmdstanpy - INFO - Chain [1] done processing
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1] done processing
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1] done processing
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1] done processing
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1] done processing
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1] done processing
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1] done processing
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1]

In [56]:
del model_info['MAPE']
best_params = model_info.iloc[0].to_dict()
model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Orders Lag 52')
#model.add_regressor('Checkout')
model.fit(Prophet_orders_data_train_log)
forecast = model.predict(future)
Prophet_train_predictions = np.exp(forecast[forecast['ds'] <= Prophet_orders_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])
Prophet_test_predictions = np.exp(forecast[forecast['ds'] > Prophet_orders_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])

# model validation plot for Orders Prophet

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_orders_data_train['ds'][5:], y=Prophet_orders_data_train['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_orders_data_train['ds'][5:], y=Prophet_train_predictions[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=Prophet_orders_data_test['ds'], y=Prophet_orders_data_test['y'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=Prophet_orders_data_test['ds'], y=Prophet_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Orders Prophet Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

15:38:45 - cmdstanpy - INFO - Chain [1] start processing
15:38:45 - cmdstanpy - INFO - Chain [1] done processing


In [57]:
# produce error metrics for Orders Prophet 

Prophet_RMSE = np.sqrt(np.mean((Prophet_test_predictions.values - Prophet_orders_data_test['y'].values)**2))

Prophet_MAPE = np.mean(abs((Prophet_orders_data_test['y'].values - Prophet_test_predictions.values)/Prophet_orders_data_test['y'].values)) * 100

Prophet_TotalError = (sum(Prophet_test_predictions[:-1].values) - sum(Prophet_orders_data_test['y'].values))
Prophet_TotalErrorPercent = abs((Prophet_TotalError / sum(Prophet_orders_data_test['y'].values)) * 100)

metrics = [['Prophet', 'Orders', round(Prophet_RMSE), "{:.2%}".format(Prophet_MAPE / 100), "{:.2%}".format(Prophet_TotalErrorPercent / 100)]]
Prophet_OrdersMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

Prophet_Orders_Train_Fit = Prophet_train_predictions
Prophet_Orders_Test_Fit = Prophet_test_predictions

total_forecast_period = 52 + 22

topline_orders_forecast_set = pd.date_range(start=Prophet_orders_data_test['ds'].iloc[-1], periods=(total_forecast_period - 22), freq='W-MON')
topline_orders_forecast_set = topline_orders_forecast_set[1:]

Prophet_orders_data_log = Prophet_orders_data.copy()
Prophet_orders_data_log[['y', 'Orders Lag 52', 'Checkout']] = log_transform(Prophet_orders_data_log[['y', 'Orders Lag 52', 'Checkout']])

model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Orders Lag 52')
#model.add_regressor('Checkout')
model.fit(Prophet_orders_data_log)
future = model.make_future_dataframe(periods = (total_forecast_period - 22 - 1), freq = 'W-MON')

orders_lag52_exog_indices = topline_orders_forecast_set.shift(-52)
orders_lag52_exog_values = Prophet_orders_data[Prophet_orders_data['ds'].isin(orders_lag52_exog_indices)]['y'].values

forecast_exog = pd.Series(data = orders_lag52_exog_values, index = topline_orders_forecast_set).to_frame().rename(columns={0: 'Orders Lag 52'})
forecast_exog['Checkout'] = Prophet_Checkout_Forecast_WithoutExtension.values
forecast_exog = log_transform(forecast_exog)
future['Orders Lag 52'] = Prophet_orders_data_log['Orders Lag 52'].append(forecast_exog['Orders Lag 52']).reset_index(drop = True)
future['Checkout'] = Prophet_checkout_data_log['y'].append(forecast_exog['Checkout']).reset_index(drop = True)

fitted = model.predict()
forecast = model.predict(future)

# dampen the forecast to constrain extrapolating prophet trend 

dampened_forecast = dampen_prophet(Prophet_orders_data_log['y'].values, fitted, forecast)
full_train = forecast.head(len(forecast) - (total_forecast_period - 22 - 1))
forecast['yhat_damp'] = full_train['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
forecast['yhat'] = np.exp(forecast['yhat'])
forecast['yhat_damp'] = np.exp(forecast['yhat_damp'])
full_train['yhat'] = np.exp(full_train['yhat'])

Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast.tail(total_forecast_period - 22 - 1)['yhat_damp']
Prophet_Orders_Full_Fit = Prophet_fits_results

# plot one year out Orders prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_orders_data['ds'][5:], y=Prophet_orders_data['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_orders_data['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_orders_forecast_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Orders Prophet Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

Prophet_Orders_Forecast_WithoutExtension = Prophet_forecast_predictions

15:38:45 - cmdstanpy - INFO - Chain [1] start processing
15:38:46 - cmdstanpy - INFO - Chain [1] done processing


In [58]:
# create forecast extension into FY'24

orders_forecast = pd.Series(data = Prophet_forecast_predictions.values, index = topline_orders_forecast_set).to_frame().rename(columns={0: 'y'})
Prophet_Orders_Full_Fit = Prophet_fits_results
orders_forecast2 = orders_forecast.join(forecast_exog)
orders_forecast2 = orders_forecast2.reset_index().rename(columns={'index': 'ds'})
orders_forecast2['Orders Lag 52'] = np.exp(orders_forecast2['Orders Lag 52'])
topline_orders_with_forecast = Prophet_orders_data.append(orders_forecast2)
topline_orders_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 51), freq = 'W-MON')
topline_orders_forecast_extension_set = topline_orders_forecast_extension_set[1:]
topline_orders_with_forecast_log = topline_orders_with_forecast.copy()
topline_orders_with_forecast_log[['y', 'Orders Lag 52', 'Checkout']] = log_transform(topline_orders_with_forecast_log[['y', 'Orders Lag 52', 'Checkout']])
model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Orders Lag 52')
#model.add_regressor('Checkout')
model.fit(topline_orders_with_forecast_log)
future = model.make_future_dataframe(periods = total_forecast_period - 52, freq = 'W-MON')
orders_lag52_exog_indices = topline_orders_forecast_extension_set.shift(-52)
orders_lag52_exog_values = topline_orders_with_forecast[topline_orders_with_forecast['ds'].isin(orders_lag52_exog_indices)]['y'].values
forecast_exog = pd.Series(data = orders_lag52_exog_values, index = topline_orders_forecast_extension_set).to_frame().rename(columns={0: 'Orders Lag 52'})

forecast_exog['Checkout'] = checkout_forecast_extension.values

forecast_exog = log_transform(forecast_exog)

future['Orders Lag 52'] = topline_orders_with_forecast_log['Orders Lag 52'].append(forecast_exog['Orders Lag 52']).reset_index(drop = True)

future['Checkout'] = topline_checkout_with_forecast_log['y'].append(forecast_exog['Checkout']).reset_index(drop = True)


fitted = model.predict()
forecast = model.predict(future)
dampened_forecast = np.exp(dampen_prophet(topline_orders_with_forecast_log['y'].values, fitted, forecast))
forecast['yhat'] = np.exp(forecast['yhat'])
full_train_endpoint = topline_orders_with_forecast['ds'].iloc[-1]
full_train = forecast[forecast['ds'] <= full_train_endpoint]
forecast['yhat_damp'] = forecast[forecast['ds'] <= full_train_endpoint]['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast[forecast['ds'] > full_train_endpoint]['yhat_damp']

15:38:46 - cmdstanpy - INFO - Chain [1] start processing
15:38:46 - cmdstanpy - INFO - Chain [1] done processing


In [59]:
# plot full FY'24 orders prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_orders_with_forecast['ds'][5:], y=topline_orders_with_forecast['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_orders_with_forecast['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_orders_forecast_extension_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
#fig.update_layout(title='Adobe.com Traffic SARIMAX Model Forecast -- AMER (G)', xaxis_title='Date', yaxis_title='No. of Actual Visits')
fig.update_layout(title='Orders Prophet Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

In [60]:
# forecast_full = topline_orders_with_forecast['y'][len(Prophet_Orders_Full_Fit):].reset_index(drop=True).append(Prophet_forecast_predictions)
# start_date = '2023-08-14'
# end_date = '2024-12-30'
# date_set = pd.date_range(start= start_date, end = end_date, freq = 'W-MON')


# fig = go.Figure()

# # Add training data to plot
# fig.add_trace(go.Scatter(x=topline_orders_with_forecast['ds'][5:], y=topline_orders_with_forecast['y'][5:len(Prophet_Orders_Full_Fit)], name= 'Train'))

# # Add training fit to plot
# fig.add_trace(go.Scatter(x=topline_orders_with_forecast['ds'][5:], y=Prophet_fits_results[5:len(Prophet_Orders_Full_Fit)], name= 'Fitted Model'))

# # Add forecast to plot
# fig.add_trace(go.Scatter(x=date_set, y=forecast_full, mode = 'lines', name= 'Forecast'))


# # Add x and y axis labels and title
# #fig.update_layout(title='Adobe.com Traffic SARIMAX Model Forecast -- AMER (G)', xaxis_title='Date', yaxis_title='No. of Actual Visits')
# fig.update_layout(title='Orders Prophet Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Orders')

# # Show plot
# fig.show()

In [61]:
# used for export of Prophet Traffic forecast

orders_forecast_extension = pd.Series(data = Prophet_forecast_predictions.values, index = topline_orders_forecast_extension_set).to_frame().rename(columns={0: 'y'})
orders_forecast = orders_forecast.append(orders_forecast_extension)
Prophet_Orders_Forecast = orders_forecast

## Ensemble

In [62]:
# aggregate results of Prophet and SARIMAX using a mean

Prophet_Orders_Train_Fit = pd.Series(Prophet_Orders_Train_Fit.values, SARIMAX_Orders_Train_Fit.index)
SARIMAX_Orders_Test_Fit = SARIMAX_Orders_Test_Fit[1:-1]
Prophet_Orders_Test_Fit = pd.Series(Prophet_Orders_Test_Fit.values, SARIMAX_Orders_Test_Fit.index)
Prophet_Orders_Full_Fit = pd.Series(Prophet_Orders_Full_Fit.values, SARIMAX_Orders_Full_Fit.index)

Ensemble_Orders_Train_Fit = (Prophet_Orders_Train_Fit + SARIMAX_Orders_Train_Fit) / 2
Ensemble_Orders_Test_Fit = (Prophet_Orders_Test_Fit + SARIMAX_Orders_Test_Fit) / 2
Ensemble_Orders_Full_Fit = (Prophet_Orders_Full_Fit + SARIMAX_Orders_Full_Fit) / 2
Ensemble_Orders_Forecast = (SARIMAX_Orders_Forecast['Orders'] + Prophet_Orders_Forecast['y']) / 2

In [63]:
# model validation plot for Orders Ensemble

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_train.index[5:], y=SARIMAX_orders_data_train['Orders'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_train.index[5:], y=Ensemble_Orders_Train_Fit.values[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_test.index, y=SARIMAX_orders_data_test['Orders'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data_test.index, y=Ensemble_Orders_Test_Fit.values, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Orders Ensemble Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

In [64]:
# output full orders error metrics

Ensemble_RMSE = np.sqrt(np.mean((Ensemble_Orders_Test_Fit.values - SARIMAX_orders_data_test['Orders'][1:].values)**2))
Emsemble_MAPE = np.mean(abs((SARIMAX_orders_data_test['Orders'][1:] - Ensemble_Orders_Test_Fit.values)/SARIMAX_orders_data_test['Orders'][1:])) * 100
Ensemble_TotalError = (sum(Ensemble_Orders_Test_Fit.values) - sum(SARIMAX_orders_data_test['Orders'][1:]))
Ensemble_TotalErrorPercent = abs((Ensemble_TotalError / sum(SARIMAX_orders_data_test['Orders'][1:])) * 100)

metrics = [['Ensemble', 'Orders', round(Ensemble_RMSE), "{:.2%}".format(Emsemble_MAPE / 100), "{:.2%}".format(Ensemble_TotalErrorPercent / 100)]]
Ensemble_OrdersMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

OrdersMetrics = SARIMAX_OrdersMetrics.append(Prophet_OrdersMetrics, ignore_index = True)
OrdersMetrics = OrdersMetrics.append(Ensemble_OrdersMetrics, ignore_index = True)
OrdersMetrics.set_index('Model', inplace = True)
OrdersMetrics

,Metric,RMSE,MAPE,Total Error
Model,,,,
SARIMAX,Orders,32231,5.31%,0.51%
Prophet,Orders,37564,7.34%,0.84%
Ensemble,Orders,32067,5.75%,2.13%


In [65]:
# plot full FY'24 orders ensemble forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data.index[5:], y=SARIMAX_orders_data['Orders'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_orders_data.index[5:], y=Ensemble_Orders_Full_Fit[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=Ensemble_Orders_Forecast.index, y=Ensemble_Orders_Forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Orders Ensemble Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Orders')

# Show plot
pyo.iplot(fig)

# Subscribers Forecasting

## SARIMAX

In [66]:
SARIMAX_subscribers_data = create_subscribers_exog_SARIMAX(SARIMAX_subscribers_data)

# split date is set arbitrarily based on current length of input data into TM1. can be made more dynamic.
split_date = '2022-12-05'

SARIMAX_subscribers_data_train = SARIMAX_subscribers_data[:split_date]
SARIMAX_subscribers_data_test = SARIMAX_subscribers_data[split_date:]

# log transform for stationarity
SARIMAX_subscribers_data_train_log = SARIMAX_subscribers_data_train.copy()
SARIMAX_subscribers_data_train_log['Subscribers'] = log_transform(SARIMAX_subscribers_data_train_log['Subscribers'])

SARIMAX_subscribers_data_train_log.index.freq = 'W-MON' # Week Start

train_endog = SARIMAX_subscribers_data_train_log['Subscribers']
train_exog = log_transform(SARIMAX_subscribers_data_train_log[['Subscribers Lag 52', 'Month Seasonal', 'Orders']])
test_endog = SARIMAX_subscribers_data_test['Subscribers']
test_exog = log_transform(SARIMAX_subscribers_data_test[['Subscribers Lag 52', 'Month Seasonal', 'Orders']])

# hyperparameter tuning with custom grid search to find optimal SARIMAX parameters 
model_info = SARIMA_grid(train_endog = train_endog,
                         train_exog = train_exog,
                         test_endog = test_endog,
                         test_exog = test_exog,
                         order = pdq, 
                         seasonal_order = PDQm)

# selects best parameters based on lowest MAPE
model_info = model_info.sort_values(by = 'MAPE')
model_info = model_info.reset_index(drop = True)

order = model_info['order'][0]
seasonal_order = model_info['seasonal_order'][0]

ARIMA_model = sm.tsa.SARIMAX(endog = train_endog, 
                             exog = train_exog,
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_model_fit = ARIMA_model.fit()
ARIMA_train_results = ARIMA_model_fit.fittedvalues

ARIMA_train_results = np.exp(ARIMA_train_results) # reverse transformation

ARIMA_test_predictions_wrapper = ARIMA_model_fit.get_prediction(start=test_endog.index[0], end=test_endog.index[-1]+timedelta(days=1),
                                                 exog = test_exog)

ARIMA_test_predictions = ARIMA_test_predictions_wrapper.predicted_mean

ARIMA_test_predictions = np.exp(ARIMA_test_predictions)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.00418D+00    |proj g|=  3.78203D-01


 This problem is unconstrained.



At iterate    5    f=  9.73366D-01    |proj g|=  1.16148D+01

At iterate   10    f= -1.33662D+00    |proj g|=  6.64738D+01

At iterate   15    f= -1.53575D+00    |proj g|=  2.08194D+01

At iterate   20    f= -1.53597D+00    |proj g|=  1.90345D+01
  ys=-3.664E-09  -gs= 2.195E-07 BFGS update SKIPPED
  ys=-9.845E-08  -gs= 1.775E-06 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     24    108      1     2     0   1.065D+01  -1.536D+00
  F =  -1.5360328247504595     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D

 This problem is unconstrained.



At iterate    5    f=  5.52021D-01    |proj g|=  1.57089D+01

At iterate   10    f= -1.21209D+00    |proj g|=  6.61596D+01

At iterate   15    f= -1.47175D+00    |proj g|=  3.79313D+00

At iterate   20    f= -1.47279D+00    |proj g|=  5.70083D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     20     76      1     0     0   5.701D+01  -1.473D+00
  F =  -1.4727864963356609     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iter

...


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           12     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.00418D+00    |proj g|=  3.77907D-01

At iterate    5    f= -4.23778D-01    |proj g|=  3.17383D+01



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   12      9     51      1     0     0   1.472D+02  -1.498D+00
  F =  -1.4980604678512874     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.00418D+00    |proj g|=  3.78230D-01

At iterate    5    f=  2.72906D-01    |proj g|=  1.88934D+01

At iterate   10    f= -1.45410D+00    |proj g|=  5.24649D+01

           * * *

Tit   = total number of iterations
Tnf   = total nu


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [67]:
# model validation plot for Subscribers SARIMAX

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_train.index[5:], y=SARIMAX_subscribers_data_train['Subscribers'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_train.index[5:], y=ARIMA_train_results[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_test.index, y=SARIMAX_subscribers_data_test['Subscribers'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_test.index, y=ARIMA_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers SARIMAX Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

In [68]:
# produce error metrics for Subscribers SARIMAX

SARIMAX_RMSE = np.sqrt(np.mean((ARIMA_test_predictions - SARIMAX_subscribers_data_test['Subscribers'])**2))

SARIMAX_MAPE = np.mean(abs((SARIMAX_subscribers_data_test['Subscribers'] - ARIMA_test_predictions[:-1])/SARIMAX_subscribers_data_test['Subscribers'])) * 100

SARIMAX_TotalError = (sum(ARIMA_test_predictions[:-1]) - sum(SARIMAX_subscribers_data_test['Subscribers']))
SARIMAX_TotalErrorPercent = abs((SARIMAX_TotalError / sum(SARIMAX_subscribers_data_test['Subscribers'])) * 100)

metrics = [['SARIMAX', 'Subscribers', round(SARIMAX_RMSE), "{:.2%}".format(SARIMAX_MAPE / 100), "{:.2%}".format(SARIMAX_TotalErrorPercent / 100)]]
SARIMAX_SubscribersMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

SARIMAX_Subscribers_Train_Fit = ARIMA_train_results
SARIMAX_Subscribers_Test_Fit = ARIMA_test_predictions

total_forecast_period = 52 + 23
topline_subscribers_forecast_set = pd.date_range(start=SARIMAX_subscribers_data_test.index[-1], periods=(total_forecast_period - 23), freq='W-MON')
topline_subscribers_forecast_set = topline_subscribers_forecast_set[1:]

# Create Exogeneous Variables for Forecast Set -- Subscribers Lag 52 and Month Seasonal, and Orders 

subscribers_lag52_exog_indices = topline_subscribers_forecast_set.shift(-52)
subscribers_lag52_exog_values = SARIMAX_subscribers_data.loc[subscribers_lag52_exog_indices]['Subscribers'].values
forecast_exog = pd.Series(data = subscribers_lag52_exog_values, index = topline_subscribers_forecast_set).to_frame().rename(columns={0: 'Subscribers Lag 52'})

month_seasonal_values = SARIMAX_subscribers_data['Month Seasonal'].unique()
roll_after_index = SARIMAX_subscribers_data['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog))
forecast_exog['Orders'] = SARIMAX_Orders_Forecast[SARIMAX_Orders_Forecast.index < orders_forecast_extension.index[0]]

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(SARIMAX_subscribers_data['Subscribers']), 
                             exog = log_transform(SARIMAX_subscribers_data[['Subscribers Lag 52', 'Month Seasonal', 'Orders']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True)
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.01752D+00    |proj g|=  3.81507D-01


 This problem is unconstrained.



At iterate    5    f=  5.44415D-01    |proj g|=  1.38044D+01

At iterate   10    f= -6.31966D-01    |proj g|=  5.28773D+01

At iterate   15    f= -1.48941D+00    |proj g|=  2.94797D+01

At iterate   20    f= -1.50880D+00    |proj g|=  3.64717D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   11     21     75      1     0     0   3.647D+01  -1.509D+00
  F =  -1.5088041636609322     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [69]:
ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog),
                                                 exog = log_transform(forecast_exog[['Subscribers Lag 52', 'Month Seasonal', 'Orders']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean
ARIMA_forecast = np.exp(ARIMA_forecast)

# create initial forecast for one full year out

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data.index[5:], y=SARIMAX_subscribers_data['Subscribers'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_subscribers_forecast_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers SARIMAX Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

In [70]:
# generate a forecast extension to go until FY'24

SARIMAX_Subscribers_Full_Fit = ARIMA_fit_results

subscribers_forecast = pd.Series(data = ARIMA_forecast.values, index = topline_subscribers_forecast_set).to_frame().rename(columns={0: 'Subscribers'})
subscribers_forecast2 = subscribers_forecast.join(forecast_exog)
topline_subscribers_with_forecast = SARIMAX_subscribers_data.append(subscribers_forecast2)
topline_subscribers_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 52), freq = 'W-MON')
topline_subscribers_forecast_extension_set = topline_subscribers_forecast_extension_set[1:]

# Create Exogeneous Variables for Forecast Extension Set -- Subscribers Lag 52, Month Seasonal, and Orders

subscribers_lag52_exog_indices = topline_subscribers_forecast_extension_set.shift(-52)
subscribers_lag52_exog_values = topline_subscribers_with_forecast.loc[subscribers_lag52_exog_indices]['Subscribers'].values
forecast_exog2 = pd.Series(data = subscribers_lag52_exog_values, index = topline_subscribers_forecast_extension_set).to_frame().rename(columns={0: 'Subscribers Lag 52'})

month_seasonal_values = forecast_exog['Month Seasonal'].unique()
roll_after_index = forecast_exog['Month Seasonal'][-1]

month_seasonal_values_rolled = roll_to_element(month_seasonal_values, roll_after_index)

forecast_exog2['Month Seasonal'] = tile_with_remainder(month_seasonal_values_rolled, len(forecast_exog2))
forecast_exog2['Orders'] = SARIMAX_Orders_Forecast[SARIMAX_Orders_Forecast.index >= orders_forecast_extension.index[0]]

ARIMA_forecast_model = sm.tsa.SARIMAX(endog = log_transform(topline_subscribers_with_forecast['Subscribers']), 
                             exog = log_transform(topline_subscribers_with_forecast[['Subscribers Lag 52', 'Month Seasonal', 'Orders']]),
                             order=order, seasonal_order=seasonal_order, 
                             mle_regression = False, time_varying_regression = True);
ARIMA_forecast_model_fit = ARIMA_forecast_model.fit()

ARIMA_fit_results = ARIMA_forecast_model_fit.fittedvalues
ARIMA_fit_results = np.exp(ARIMA_fit_results)

ARIMA_forecast_predictions_wrapper = ARIMA_forecast_model_fit.get_forecast(len(forecast_exog2),
                                                 exog = log_transform(forecast_exog2[['Subscribers Lag 52', 'Month Seasonal', 'Orders']]))

ARIMA_forecast = ARIMA_forecast_predictions_wrapper.predicted_mean

ARIMA_forecast = np.exp(ARIMA_forecast)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.03128D+00    |proj g|=  3.83700D-01

At iterate    5    f=  9.14130D-01    |proj g|=  1.04757D+01

At iterate   10    f= -1.43805D-01    |proj g|=  3.26533D+01

At iterate   15    f= -1.61157D+00    |proj g|=  5.66160D+01
  ys=-9.938E-06  -gs= 3.434E-05 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   11     19     76      1     1     0   3.965D+01  -1.644D+00
  F =  -1.6441592920230870     

CONVERGENCE: REL_


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [71]:
# create full forecast for entire FY'24

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_subscribers_with_forecast.index[5:], y=topline_subscribers_with_forecast['Subscribers'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_subscribers_with_forecast.index[5:], y=ARIMA_fit_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_subscribers_forecast_extension_set, y=ARIMA_forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers SARIMAX Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

In [72]:
# used to export SARIMAX Subscribers forecasted values

subscribers_forecast_extension = pd.Series(data = ARIMA_forecast.values, index = topline_subscribers_forecast_extension_set).to_frame().rename(columns={0: 'Subscribers'})
subscribers_forecast = subscribers_forecast.append(subscribers_forecast_extension)
SARIMAX_Subscribers_Forecast = subscribers_forecast

## Prophet

In [73]:
Prophet_subscribers_data = create_subscribers_exog_Prophet(Prophet_subscribers_data)
Prophet_subscribers_data = Prophet_subscribers_data.rename(columns={"Fiscal Date": "ds", "Subscribers": "y"})
Prophet_subscribers_data_train = Prophet_subscribers_data[Prophet_subscribers_data['ds'] <= split_date]
Prophet_subscribers_data_test = Prophet_subscribers_data[Prophet_subscribers_data['ds'] > split_date]

# log transform for stationarity
Prophet_subscribers_data_train_log = Prophet_subscribers_data_train.copy()
Prophet_subscribers_data_train_log[['y', 'Subscribers Lag 52', 'Orders']] = log_transform(Prophet_subscribers_data_train_log[['y', 'Subscribers Lag 52', 'Orders']])

model = Prophet()
model.fit(Prophet_subscribers_data_train_log)
future = model.make_future_dataframe(periods = len(Prophet_subscribers_data_test), freq = 'W-MON')
subscribers_forecast_set = pd.date_range(Prophet_subscribers_data_train_log['ds'].iloc[-1], periods = len(Prophet_subscribers_data_test) + 1, freq = 'W-MON')
subscribers_forecast_set = subscribers_forecast_set[1:]

# Create Exogeneous Variables -- Subscribers Lag 52 and Orders
subscribers_lag52_exog_indices = subscribers_forecast_set.shift(-52)
subscribers_lag52_exog_values = Prophet_subscribers_data[Prophet_subscribers_data['ds'].isin(subscribers_lag52_exog_indices)]['y'].values

forecast_exog = pd.Series(data = subscribers_lag52_exog_values, index = subscribers_forecast_set).to_frame().rename(columns={0: 'Subscribers Lag 52'})
forecast_exog['Orders'] = Prophet_orders_data_test['y'].values
forecast_exog = log_transform(forecast_exog)

future['Subscribers Lag 52'] = Prophet_subscribers_data_train_log['Subscribers Lag 52'].append(forecast_exog['Subscribers Lag 52']).reset_index(drop = True)
future['Orders'] = Prophet_orders_data_train_log['y'].append(forecast_exog['Orders']).reset_index(drop = True)

# hyperparameter tuning with custom grid search function for optimal parameter selection
model_info = Prophet_HyperparameterTuning(Prophet_subscribers_data_train_log, Prophet_subscribers_data_test['y'], future)

15:39:03 - cmdstanpy - INFO - Chain [1] start processing
15:39:03 - cmdstanpy - INFO - Chain [1] done processing
15:39:03 - cmdstanpy - INFO - Chain [1] start processing
15:39:03 - cmdstanpy - INFO - Chain [1] done processing
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1] done processing
15:39:04 - cmdstanpy - INFO - Chain [1] start processing
15:39:04 - cmdstanpy - INFO - Chain [1]

In [74]:
del model_info['MAPE']
best_params = model_info.iloc[0].to_dict()

model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Subscribers Lag 52')
model.add_regressor('Orders')
model.fit(Prophet_subscribers_data_train_log)
forecast = model.predict(future)
Prophet_train_predictions = np.exp(forecast[forecast['ds'] <= Prophet_subscribers_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])
Prophet_test_predictions = np.exp(forecast[forecast['ds'] > Prophet_subscribers_data_train['ds'].iloc[-1].strftime('%Y-%m-%d')]['yhat'])

# model validation plot for Subscribers Prophet

# Create plotly figure
fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_subscribers_data_train['ds'][5:], y=Prophet_subscribers_data_train['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_subscribers_data_train['ds'][5:], y=Prophet_train_predictions[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=Prophet_subscribers_data_test['ds'], y=Prophet_subscribers_data_test['y'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=Prophet_subscribers_data_test['ds'], y=Prophet_test_predictions, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers Prophet Model -- Fit and Predict', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

15:39:06 - cmdstanpy - INFO - Chain [1] start processing
15:39:06 - cmdstanpy - INFO - Chain [1] done processing


In [75]:
# produce error metrics for Subscribers Prophet 

Prophet_RMSE = np.sqrt(np.mean((Prophet_test_predictions.values - Prophet_subscribers_data_test['y'].values)**2))

Prophet_MAPE = np.mean(abs((Prophet_subscribers_data_test['y'].values - Prophet_test_predictions.values)/Prophet_subscribers_data_test['y'].values)) * 100

Prophet_TotalError = (sum(Prophet_test_predictions[:-1].values) - sum(Prophet_subscribers_data_test['y'].values))
Prophet_TotalErrorPercent = abs((Prophet_TotalError / sum(Prophet_subscribers_data_test['y'].values)) * 100)

metrics = [['Prophet', 'Subscribers', round(Prophet_RMSE), "{:.2%}".format(Prophet_MAPE / 100), "{:.2%}".format(Prophet_TotalErrorPercent / 100)]]
Prophet_SubscribersMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

Prophet_Subscribers_Train_Fit = Prophet_train_predictions
Prophet_Subscribers_Test_Fit = Prophet_test_predictions

total_forecast_period = 52 + 22

topline_subscribers_forecast_set = pd.date_range(start=Prophet_subscribers_data_test['ds'].iloc[-1], periods=(total_forecast_period - 22), freq='W-MON')
topline_subscribers_forecast_set = topline_subscribers_forecast_set[1:]

Prophet_subscribers_data_log = Prophet_subscribers_data.copy()
Prophet_subscribers_data_log[['y', 'Subscribers Lag 52', 'Orders']] = log_transform(Prophet_subscribers_data_log[['y', 'Subscribers Lag 52', 'Orders']])

model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Subscribers Lag 52')
model.add_regressor('Orders')
model.fit(Prophet_subscribers_data_log)
future = model.make_future_dataframe(periods = (total_forecast_period - 22 - 1), freq = 'W-MON')

subscribers_lag52_exog_indices = topline_subscribers_forecast_set.shift(-52)
subscribers_lag52_exog_values = Prophet_subscribers_data[Prophet_subscribers_data['ds'].isin(subscribers_lag52_exog_indices)]['y'].values

forecast_exog = pd.Series(data = subscribers_lag52_exog_values, index = topline_subscribers_forecast_set).to_frame().rename(columns={0: 'Subscribers Lag 52'})
forecast_exog['Orders'] = Prophet_Orders_Forecast_WithoutExtension.values
forecast_exog = log_transform(forecast_exog)
future['Subscribers Lag 52'] = Prophet_subscribers_data_log['Subscribers Lag 52'].append(forecast_exog['Subscribers Lag 52']).reset_index(drop = True)
future['Orders'] = Prophet_orders_data_log['y'].append(forecast_exog['Orders']).reset_index(drop = True)

fitted = model.predict()
forecast = model.predict(future)

# dampen the forecast to constrain extrapolating prophet trend 

dampened_forecast = dampen_prophet(Prophet_subscribers_data_log['y'].values, fitted, forecast)
full_train = forecast.head(len(forecast) - (total_forecast_period - 22 - 1))
forecast['yhat_damp'] = full_train['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
forecast['yhat'] = np.exp(forecast['yhat'])
forecast['yhat_damp'] = np.exp(forecast['yhat_damp'])
full_train['yhat'] = np.exp(full_train['yhat'])

Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast.tail(total_forecast_period - 22 - 1)['yhat_damp']
Prophet_Subscribers_Full_Fit = Prophet_fits_results

# plot one year out Subscribers prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=Prophet_subscribers_data['ds'][5:], y=Prophet_subscribers_data['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=Prophet_subscribers_data['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_subscribers_forecast_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers Prophet Model -- Fit & Forecast', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

15:39:06 - cmdstanpy - INFO - Chain [1] start processing
15:39:06 - cmdstanpy - INFO - Chain [1] done processing


In [76]:
# create forecast extension into FY'24

subscribers_forecast = pd.Series(data = Prophet_forecast_predictions.values, index = topline_subscribers_forecast_set).to_frame().rename(columns={0: 'y'})
Prophet_Subscribers_Full_Fit = Prophet_fits_results
subscribers_forecast2 = subscribers_forecast.join(forecast_exog)
subscribers_forecast2 = subscribers_forecast2.reset_index().rename(columns={'index': 'ds'})
subscribers_forecast2['Subscribers Lag 52'] = np.exp(subscribers_forecast2['Subscribers Lag 52'])
topline_subscribers_with_forecast = Prophet_subscribers_data.append(subscribers_forecast2)

topline_subscribers_forecast_extension_set = pd.date_range(forecast_exog.index[-1], periods = (total_forecast_period - 51), freq = 'W-MON')
topline_subscribers_forecast_extension_set = topline_subscribers_forecast_extension_set[1:]

topline_subscribers_with_forecast_log = topline_subscribers_with_forecast.copy()
topline_subscribers_with_forecast_log[['y', 'Subscribers Lag 52', 'Orders']] = log_transform(topline_subscribers_with_forecast_log[['y', 'Subscribers Lag 52', 'Orders']])
model = Prophet(**best_params)
model.add_seasonality(name = 'monthly', period = 4, fourier_order = 5)
model.add_regressor('Subscribers Lag 52')
model.add_regressor('Orders')

model.fit(topline_subscribers_with_forecast_log)
future = model.make_future_dataframe(periods = total_forecast_period - 52, freq = 'W-MON')
subscribers_lag52_exog_indices = topline_subscribers_forecast_extension_set.shift(-52)
subscribers_lag52_exog_values = topline_subscribers_with_forecast[topline_subscribers_with_forecast['ds'].isin(subscribers_lag52_exog_indices)]['y'].values
forecast_exog = pd.Series(data = subscribers_lag52_exog_values, index = topline_subscribers_forecast_extension_set).to_frame().rename(columns={0: 'Subscribers Lag 52'})

forecast_exog['Orders'] = orders_forecast_extension.values

forecast_exog = log_transform(forecast_exog)

future['Subscribers Lag 52'] = topline_subscribers_with_forecast_log['Subscribers Lag 52'].append(forecast_exog['Subscribers Lag 52']).reset_index(drop = True)

future['Orders'] = topline_orders_with_forecast_log['y'].append(forecast_exog['Orders']).reset_index(drop = True)


fitted = model.predict()
forecast = model.predict(future)
dampened_forecast = np.exp(dampen_prophet(topline_subscribers_with_forecast_log['y'].values, fitted, forecast))
forecast['yhat'] = np.exp(forecast['yhat'])
full_train_endpoint = topline_subscribers_with_forecast['ds'].iloc[-1]
full_train = forecast[forecast['ds'] <= full_train_endpoint]
forecast['yhat_damp'] = forecast[forecast['ds'] <= full_train_endpoint]['yhat'].append(pd.Series(dampened_forecast)).reset_index(drop=True)
Prophet_fits_results = full_train['yhat']
Prophet_forecast_predictions = forecast[forecast['ds'] > full_train_endpoint]['yhat_damp']

15:39:06 - cmdstanpy - INFO - Chain [1] start processing
15:39:06 - cmdstanpy - INFO - Chain [1] done processing


In [77]:
# plot full FY'24 subscribers prophet forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=topline_subscribers_with_forecast['ds'][5:], y=topline_subscribers_with_forecast['y'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=topline_subscribers_with_forecast['ds'][5:], y=Prophet_fits_results[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=topline_subscribers_forecast_extension_set, y=Prophet_forecast_predictions, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers Prophet Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

In [78]:
# used for export of Prophet Subscribers forecast

subscribers_forecast_extension = pd.Series(data = Prophet_forecast_predictions.values, index = topline_subscribers_forecast_extension_set).to_frame().rename(columns={0: 'y'})
subscribers_forecast = subscribers_forecast.append(subscribers_forecast_extension)
Prophet_Subscribers_Forecast = subscribers_forecast

## Ensemble

In [79]:
# aggregate results of Prophet and SARIMAX using a mean

Prophet_Subscribers_Train_Fit = pd.Series(Prophet_Subscribers_Train_Fit.values, SARIMAX_Subscribers_Train_Fit.index)
SARIMAX_Subscribers_Test_Fit = SARIMAX_Subscribers_Test_Fit[1:-1]
Prophet_Subscribers_Test_Fit = pd.Series(Prophet_Subscribers_Test_Fit.values, SARIMAX_Subscribers_Test_Fit.index)
Prophet_Subscribers_Full_Fit = pd.Series(Prophet_Subscribers_Full_Fit.values, SARIMAX_Subscribers_Full_Fit.index)

Ensemble_Subscribers_Train_Fit = (Prophet_Subscribers_Train_Fit + SARIMAX_Subscribers_Train_Fit) / 2
Ensemble_Subscribers_Test_Fit = (Prophet_Subscribers_Test_Fit + SARIMAX_Subscribers_Test_Fit) / 2
Ensemble_Subscribers_Full_Fit = (Prophet_Subscribers_Full_Fit + SARIMAX_Subscribers_Full_Fit) / 2
Ensemble_Subscribers_Forecast = (SARIMAX_Subscribers_Forecast['Subscribers'] + Prophet_Subscribers_Forecast['y']) / 2

In [80]:
# model validation plot for Subscribers Ensemble

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_train.index[5:], y=SARIMAX_subscribers_data_train['Subscribers'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_train.index[5:], y=Ensemble_Subscribers_Train_Fit.values[5:], name= 'Fitted Model'))

# Add testing data to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_test.index, y=SARIMAX_subscribers_data_test['Subscribers'], name= 'Test'))

# Add testing fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data_test.index, y=Ensemble_Subscribers_Test_Fit.values, name= 'Model Predict'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers Ensemble Model -- Fit & Predict', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

In [81]:
# output full subscribers error metrics

Ensemble_RMSE = np.sqrt(np.mean((Ensemble_Subscribers_Test_Fit.values - SARIMAX_subscribers_data_test['Subscribers'][1:].values)**2))
Emsemble_MAPE = np.mean(abs((SARIMAX_subscribers_data_test['Subscribers'][1:] - Ensemble_Subscribers_Test_Fit.values)/SARIMAX_subscribers_data_test['Subscribers'][1:])) * 100
Ensemble_TotalError = (sum(Ensemble_Subscribers_Test_Fit.values) - sum(SARIMAX_subscribers_data_test['Subscribers'][1:]))
Ensemble_TotalErrorPercent = abs((Ensemble_TotalError / sum(SARIMAX_subscribers_data_test['Subscribers'][1:])) * 100)

metrics = [['Ensemble', 'Subscribers', round(Ensemble_RMSE), "{:.2%}".format(Emsemble_MAPE / 100), "{:.2%}".format(Ensemble_TotalErrorPercent / 100)]]
Ensemble_SubscribersMetrics = pd.DataFrame(metrics, columns = ['Model', 'Metric', 'RMSE', 'MAPE', 'Total Error'])

SubscribersMetrics = SARIMAX_SubscribersMetrics.append(Prophet_SubscribersMetrics, ignore_index = True)
SubscribersMetrics = SubscribersMetrics.append(Ensemble_SubscribersMetrics, ignore_index = True)
SubscribersMetrics.set_index('Model', inplace = True)
SubscribersMetrics

,Metric,RMSE,MAPE,Total Error
Model,,,,
SARIMAX,Subscribers,16156,6.13%,3.68%
Prophet,Subscribers,10526,3.39%,3.96%
Ensemble,Subscribers,12406,4.37%,2.49%


In [82]:
# plot full FY'24 subscribers ensemble forecast

fig = go.Figure()

# Add training data to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data.index[5:], y=SARIMAX_subscribers_data['Subscribers'][5:], name= 'Train'))

# Add training fit to plot
fig.add_trace(go.Scatter(x=SARIMAX_subscribers_data.index[5:], y=Ensemble_Subscribers_Full_Fit[5:], name= 'Fitted Model'))

# Add forecast to plot
fig.add_trace(go.Scatter(x=Ensemble_Subscribers_Forecast.index, y=Ensemble_Subscribers_Forecast, mode = 'lines', name= 'Forecast'))


# Add x and y axis labels and title
fig.update_layout(title='Subscribers Ensemble Model -- Fit & Forecast FY24', xaxis_title='Date', yaxis_title='No. of Subscribers')

# Show plot
pyo.iplot(fig)

# Final Results and Exports

In [83]:
# concatenate all metrics into one single view

TrafficMetrics.reset_index(inplace = True)
CheckoutMetrics.reset_index(inplace = True)
OrdersMetrics.reset_index(inplace = True)
SubscribersMetrics.reset_index(inplace = True)

TotalMetrics = TrafficMetrics.append(CheckoutMetrics).append(OrdersMetrics).append(SubscribersMetrics)

In [84]:
TotalMetrics.set_index(['Metric', 'Model'])

RMSE   MAPE Total Error
Metric      Model                               
Traffic     SARIMAX   2588565  6.65%       4.22%
            Prophet   1616788  3.83%       4.86%
            Ensemble  1986678  4.95%       3.20%
Checkout    SARIMAX    170347  5.01%       0.27%
            Prophet    227163  6.65%       1.70%
            Ensemble   166602  5.02%       2.47%
Orders      SARIMAX     32231  5.31%       0.51%
            Prophet     37564  7.34%       0.84%
            Ensemble    32067  5.75%       2.13%
Subscribers SARIMAX     16156  6.13%       3.68%
            Prophet     10526  3.39%       3.96%
            Ensemble    12406  4.37%       2.49%

In [85]:
FiscalCalendar = pd.read_csv('/Users/adulla/Desktop/Adobe_FiscalCalendar.csv')

In [86]:
temp = FiscalCalendar[FiscalCalendar['Fiscal WeekinYear'] > '2023-32'][['Fiscal WeekinYear']]
temp = temp.drop_duplicates()
Ensemble_Traffic_Forecast.index = temp['Fiscal WeekinYear'][:len(Ensemble_Traffic_Forecast)].values

In [87]:
export = Ensemble_Traffic_Forecast.to_frame().rename(columns = {0: "Traffic"})
export['Checkout'] = Ensemble_Checkout_Forecast.values
export['Orders'] = Ensemble_Orders_Forecast.values
export['Subscribers'] = Ensemble_Subscribers_Forecast.values
export['Geo'] = geo

In [88]:
# goes into 2025-01. Cut last value.

export = export.iloc[:-1, :]

In [89]:
export

,Traffic,Checkout,Orders,Subscribers,Geo
2023-33,31798170.243,2758650.603,422079.084,221236.660,ALL (G)
2023-34,31564160.708,2776554.668,426429.898,223246.344,ALL (G)
2023-35,32405779.794,2856442.322,427192.493,226954.774,ALL (G)
2023-36,32287771.292,2850597.679,426665.912,222600.748,ALL (G)
2023-37,33063227.732,3022578.854,431690.498,221430.253,ALL (G)
...,...,...,...,...,...
2024-48,36303661.777,3539028.918,471961.453,242822.950,ALL (G)
2024-49,36732133.369,3695114.781,467037.475,239525.607,ALL (G)
2024-50,37089869.468,3787037.387,469462.168,247924.186,ALL (G)
2024-51,37332393.543,3821477.860,475464.808,259908.929,ALL (G)


In [90]:
#export.to_csv('/Users/adulla/Desktop/Final_Build/ALL_Forecast.csv')